In [ ]:
# Install required packages
!pip install -q google-cloud-storage google-cloud-aiplatform google-api-core
!pip install --upgrade google-api-core
!pip install --upgrade google-cloud-aiplatform

# Import required packages and authenticate
import os
import csv
import json
import pandas as pd
import io
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part
from google.cloud import storage
from google.colab import auth
import pandas as pd
import json
from datetime import datetime
import random
from typing import Dict, List, Any, Optional

import sys
maxInt = sys.maxsize

while True:
    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

# Authenticate
auth.authenticate_user()

# Initialize Vertex AI
vertexai.init(project='label-studio-424123', location='us-central1')

class DialogueStyleGenerator:
    def __init__(self):
        self.approach_types = {
            'INFORMATION_GATHERING': [
                'Summary questions',
                'Clarification questions',
                'Exploratory Questions',
                'Process questions',
                'Story questions',
                'What-If Scenarios',
                'Fact-checking questions',
                'Definition questions',
                'Example questions',
                'Source analysis questions'
            ],
            'CRITICAL_THINKING': [
                'Analysis questions',
                'Critical questions',
                'Challenge questions',
                'Hypothetical questions',
                'Cause-and-effect questions',
                'Contradiction Exploring Questions',
                'Interpretation questions',
                'Comparison questions',
                'Assumption questions',
                'Bias detection questions',
                'Logical fallacy questions',
                'Evidence evaluation questions'
            ],
            'REFLECTIVE_AND_APPLICATION': [
                'Connection questions',
                'Pattern questions',
                'Perspective questions',
                'Evaluation questions',
                'Application questions',
                'Reflective questions',
                'Decision-Making Questions',
                'Empathy-Based Questions',
                'Ethical Dilemmas',
                'Conceptual Questions',
                'Predictive Questions',
                'Sequential Questions',
                'Validation Questions',
                'Relevance-Based Questions',
                'Feasibility Questions',
                'Insight-Seeking Questions',
                'Goal prioritization questions',
                'Long-term impact questions',
                'Alternative solutions questions',
                'Consequence exploration questions'
            ],
            'CREATIVE_EXPLORATION': [
                'Imagination questions',
                'Brainstorming questions',
                'Role-playing questions',
                'Alternative possibilities questions',
                'Future visioning questions',
                'Idea generation questions',
                'Innovation questions',
                'Creative solution questions',
                'Out-of-the-box thinking questions'
            ],
            'PROBLEM_SOLVING': [
                'Problem-posing questions',
                'Solution-finding questions',
                'Consequence analysis questions',
                'Action planning questions',
                'Obstacle identification questions',
                'Root cause analysis questions',
                'Risk assessment questions',
                'Mitigation strategy questions',
                'Decision tree questions'
            ],
            'ETHICAL_CONSIDERATIONS': [
                'Values clarification questions',
                'Moral dilemma questions',
                'Animal rights questions',
                'Sentience impact questions',
                'Long-term impact questions',
                'Speciesism questions',
                'Anthropocentrism questions',
                'Environmental ethics questions',
                'Social justice questions'
            ],
            'AGENT_ACTION': [
                'Content generation instructions',
                'Social media interaction instructions',
                'Email communication instructions',
                'Data analysis and prediction instructions',
                'Real-world task automation instructions',
                'Multi-agent coordination instructions',
                'Autonomous learning and adaptation instructions',
                'Personalized user interaction instructions',
                'Report generation instructions',
                'Security and privacy instructions',
                'Error handling and recovery instructions',
                'Explainable action instructions',
                'Tool-based research instructions',
                'Data visualization instructions',
                'API call instructions',
                'Code interpreter instructions',
                'Search instructions',
                'RAG instructions',
                'Advanced data analysis instructions',
                'Simulation and modeling instructions',
                'Natural language processing instructions',
                'Website interaction instructions',
                'Financial transaction instructions',
                'Legal document analysis instructions',
                'Translation instructions',
                'Sentiment analysis instructions'
            ]
        }
        self.response_techniques = {
            'DIRECT_STATEMENTS': [
                'Connect ideas',
                'Summarize briefly',
                'Compare items',
                'Tell stories',
                'Draw analogies',
                'Analyze contexts',
                'Map problems',
                'Explore emotions',
                'Provide examples',
                'Offer explanations',
                'Clarify concepts',
                'Define terms',
                'State opinions',
                'Make predictions',
                'Offer solutions',
                'Give advice',
                'Share experiences',
                'Express empathy',
                'Justify Reasoning',
                'State Assumptions',
                'Point out Contrasts',
                'Share Historical Insights',
                'Highlight Similarities',
                'Summarize Perspectives',
                'Identify Priorities',
                'Provide Contextual Background',
                'Emphasize Key Details',
                'Offer Plausible Outcomes',
                'Simplify Complex Topics',
                'Describe Best Practices'
            ],
            'CONNECTION_BASED': [
                'Explaining Connections',
                'Cause and Effect',
                'Interdisciplinary Links',
                'Pattern Recognition',
                'Historical Comparison',
                'Analogical Reasoning',
                'Identifying Relationships',
                'Tracing Influences',
                'Network Mapping',
                'Synthesising Perspectives',
                'Exploring Systems Interdependencies',
                'Cross-Cultural Comparisons',
                'Mapping Ecosystem Dynamics',
                'Feedback Loop Analysis',
                'Correlational Studies',
                'Tracing Evolution of Ideas',
                'Comparing Frameworks',
                'Systemic Thinking',
                'Meta-Analysis',
                'Linking Abstract Concepts',
                'Exploring Underlying Mechanisms',
                'Web of Influence Analysis',
                'Exploring Domino Effects',
                'Temporal Sequencing',
                'Cross-Disciplinary Narratives'
            ],
            'ANALYTICAL': [
                'Comparative Analysis',
                'Critical Evaluation',
                'Problem-Solution Mapping',
                'Gap Analysis',
                'Hypothesis Testing',
                'Contradiction Spotting',
                'Root Cause Analysis',
                'SWOT Analysis',
                'Decision Analysis',
                'Risk Assessment',
                'Logical Reasoning',
                'Evidence Evaluation',
                'Quantitative Comparisons',
                'Scenario Analysis',
                'Prioritization Matrices',
                'Cost-Benefit Analysis',
                'Trend Analysis',
                'Counterfactual Reasoning',
                'Sensitivity Analysis',
                'Exploring Trade-offs',
                'Data Interpretation',
                'Probability Assessment',
                'Error Analysis',
                'Factor Analysis',
                'Causation vs. Correlation Exploration',
                'Decision Tree Evaluation'
            ],
            'SUMMARY': [
                'TLDR Summaries',
                'Extreme Summaries',
                'Key Takeaways',
                'Topical Breakdown',
                'Bullet Point Summaries',
                'Abstract Writing',
                'Executive Summaries',
                'Visual Summaries',
                'Simplified Explanations',
                'Condensing Information',
                'Micro-Summaries',
                'Quick Recap',
                'Concise Overviews',
                'Snapshot Summaries',
                'Content Mapping',
                'Summarizing Themes',
                'Highlighting Core Insights',
                'Layered Summaries',
                'Chapter Recaps',
                'Summary Tables',
                'Dashboards of Information',
                'Condensed Narratives',
                'Synopsis Creation',
                'Prioritized Highlights',
                'Simplified Comparisons'
            ],
            'STRUCTURAL': [
                'Step-by-Step Instructions',
                'Question-Based Dissection',
                'Chronological Analysis',
                'Layered Reading',
                'Hierarchical Organization',
                'Flowcharting',
                'Mind Mapping',
                'Deconstruction',
                'Reverse Engineering',
                'Process Analysis',
                'Mapping Dependencies',
                'Component Breakdown',
                'Workflow Optimization',
                'Pipeline Structuring',
                'Framework Development',
                'Blueprint Mapping',
                'Organizational Design',
                'Taxonomy Building',
                'Schematic Outlines',
                'Crosswalk Diagrams',
                'Scaffold Structuring',
                'Diagrammatic Reasoning',
                'Process Benchmarking',
                'Structure Versus Function Analysis',
                'Pathway Modeling'
            ],
            'PERSPECTIVE': [
                'Point-of-View Analysis',
                'Counterarguments',
                'Contextual Analysis',
                'Emotion-Based Analysis',
                'Ethical Considerations',
                'Bias Detection',
                'Empathy Mapping',
                'Audience Analysis',
                'Cultural Interpretation',
                'Social Impact Assessment',
                'Stakeholder Mapping',
                'Historical Lenses',
                'Personal Narratives',
                'Multiple Viewpoints',
                'Examining Underlying Assumptions',
                'Futuristic Perspectives',
                'Ideological Critique',
                'Global versus Local Perspectives',
                'Intersectional Analysis',
                'Moral Philosophy Integration',
                'Nuance Exploration',
                'Perspective Shifts',
                'Diversity and Inclusion Analysis',
                'Value-Based Analysis',
                'Comparing Worldviews'
            ],
            'CONCEPTUAL': [
                'Analogies',
                'Visualizing Concepts',
                'Definitional Responses',
                'Hypothetical Scenarios',
                'Thought Experiments',
                'Abstraction',
                'Generalization',
                'Concept Mapping',
                'Model Building',
                'Philosophical Inquiry',
                'Conceptual Framework Development',
                'Theory Crafting',
                'Exploring Paradoxes',
                'Scenario Planning',
                'Exploring Principles',
                'Metacognitive Reflection',
                'Integrating Diverse Concepts',
                'Devising Taxonomies',
                'Exploring Paradigms',
                'Unpacking Complexity',
                'Categorical Analysis',
                'Building Mental Models',
                'Defining Key Constructs',
                'Conceptual Blending',
                'Cross-Concept Analysis'
            ],
            'INTERPRETATIVE': [
                'Thematic Exploration',
                'Inference Drawing',
                'Defining Assumptions',
                'Summarizing Subtexts',
                'Symbolism Analysis',
                'Metaphor Interpretation',
                'Reading Between the Lines',
                'Deciphering Meaning',
                'Hermeneutics',
                'Semiotic Analysis',
                'Narrative Deconstruction',
                'Layering Interpretations',
                'Contextual Symbolism',
                'Tone and Mood Analysis',
                'Implied Meaning Identification',
                'Cross-Textual Interpretation',
                'Literary Critique',
                'Discourse Analysis',
                'Cultural Semiotics',
                'Philosophical Deconstruction',
                'Juxtaposition Analysis',
                'Diving into Subconscious Themes',
                'Analyzing Motifs',
                'Identifying Allegorical Layers',
                'Decoding Visual Language'
            ],
            'PRACTICAL': [
                'Application to Real Life',
                'Pros and Cons',
                'Why and How',
                'Problem-Solution',
                'Case Studies',
                'Best Practices',
                'Action Planning',
                'Decision Making',
                'Troubleshooting',
                'Implementation Strategies',
                'Developing Playbooks',
                'Feasibility Analysis',
                'Operational Frameworks',
                'Policy Recommendations',
                'Step-by-Step Proposals',
                'Prototyping Solutions',
                'Evaluating Effectiveness',
                'Real-World Examples',
                'Use Case Development',
                'Process Adaptation',
                'Testing Approaches',
                'Guideline Formulation',
                'Mitigation Strategies',
                'Design Thinking Applications',
                'Scalable Solutions'
            ],
            'NARRATIVE': [
                'Storytelling Q&A',
                'Linguistic Analysis',
                'Socratic Questioning',
                'Explaining Implications',
                'Character Analysis',
                'Plot Analysis',
                'Narrative Structure',
                'Creative Writing',
                'Worldbuilding',
                'Rhetorical Analysis',
                'Exploring Narratives',
                'Framing Stories',
                'Plot Dynamics Mapping',
                'Subplot Identification',
                'Story Arc Evaluation',
                'Hero’s Journey Analysis',
                'Prose Dissection',
                'Dialogue Interpretation',
                'Thematic Storytelling',
                'Foreshadowing Exploration',
                'Conflict Resolution in Stories',
                'Cohesion Analysis',
                'Symbolic Narratives',
                'Exploring Narrator Bias',
                'Dynamic World Creation'
            ]
        }

    def generate_dialogue_style(self):
        """Generate a random combination of dialogue styles"""

        # Pick 1-2 approaches overall
        all_approaches = [
            approach for approaches in self.approach_types.values()
            for approach in approaches
        ]
        selected_approaches = []
        if all_approaches:
            num_approaches = random.randint(1, 2)
            selected_approaches = random.sample(all_approaches, min(num_approaches, len(all_approaches)))

        # Pick 1-2 techniques overall
        all_techniques = [
            technique for techniques in self.response_techniques.values()
            for technique in techniques
        ]
        selected_techniques = []
        if all_techniques:
            num_techniques = random.randint(1, 2)
            selected_techniques = random.sample(all_techniques, min(num_techniques, len(all_techniques)))

        return {
            'approaches': selected_approaches,
            'techniques': selected_techniques,
            'style_tags': selected_approaches + selected_techniques
        }

class PersonaGenerator:
    def __init__(self):
        self.human_roles = [
            # Political
            'Minister of Traditional Agriculture and Livestock',
            'Senator Opposing Plant-Based Initiatives',
            'Lobbyist for Industrial Animal Agriculture',
            'Policy Director Fighting Vegan Labeling Laws',
            'Rural Representative Defending Animal-Based Industries',
            'Minister for Plant-Based Food Development',
            'Politician Balancing Agricultural Stakeholder Interests',
            'Rural Representative and Livestock Farmer',
            'City Council Member Advocating Vegan Infrastructure',
            'State Senator Exploring Sustainable Farming Practices',
            'National Representative Supporting Livestock Trade',
            'Agricultural Policy Advisor Focusing on Alternative Proteins',
            'Legislative Consultant for Traditional Ranching Interests',
            'Urban Agriculture Committee Chair',
            'Food Security Policy Director',
            'Regional Development Officer for Agricultural Innovation',
            'Parliamentary Secretary for Alternative Protein Research',
            'Local Food Systems Coordinator',
            'Indigenous Food Sovereignty Advocate',
            'International Trade Representative for Agricultural Products',
            'Commissioner of Sustainable Agriculture',

                        # Military & Defense
            'Military Catering Contract Manager',
            'Defense Procurement Officer for Leather Goods',
            'Military Equipment Standards Director',
            'Armed Forces Nutrition Policy Director',
            'Military Base Food Systems Manager',
            'Defense Logistics Food Chain Director',
            'Military Uniform Materials Specialist',
            'Combat Ration Development Manager',
            'Military Sustainability Program Director',
            'Defense Supplier Certification Manager',
            'Military Animal Use Program Director',
            'Armed Forces Menu Planning Coordinator',
            'Military Food Innovation Specialist',
            'Defense Materials Research Director',
            'Military Environmental Impact Manager',
            'Defense Supply Chain Transformation Director',

            # Financial Regulators
            'Agricultural Commodities Exchange Regulator',
            'Food Industry Investment Oversight Director',
            'Agricultural Futures Trading Commissioner',
            'Livestock Market Compliance Officer',
            'Food System Credit Rating Agency Director',
            'Agricultural Bank Examiner',
            'Food Industry Securities Regulator',
            'Farm Credit System Oversight Manager',
            'Agricultural Market Trading Supervisor',
            'Food Industry Financial Auditor',
            'Sustainable Investment Standards Director',
            'Agricultural Derivatives Trading Monitor',
            'Food System Risk Assessment Director',
            'Market Manipulation Investigation Manager',
            'Agricultural Finance Ethics Officer',
            'Sustainable Finance Framework Developer',

            # Legal & Regulatory Enforcement
            'Animal Industry Lobbyist',
            'Agricultural Patent Examiner',
            'Food Standards Agency Director',
            'Agricultural Trade Agreement Negotiator',
            'Meat Inspection Standards Director',
            'Food Labeling Enforcement Officer',
            'Animal Transport Regulations Manager',
            'Slaughterhouse Compliance Director',
            'Agricultural Practice Enforcement Chief',
            'Food Safety Prosecution Coordinator',
            'Animal Welfare Standards Enforcer',
            'Agricultural Policy Enforcement Director',
            'Food Industry Compliance Investigator',
            'Farm Practice Audit Manager',
            'Industry Standards Enforcement Officer',
            'Regulatory Framework Development Director',

            # Venue Management & Operations
            'Stadium Food Service Director',
            'Arena Concessions Policy Manager',
            'Sports Venue Menu Planning Director',
            'Entertainment Complex Food Operations Manager',
            'Large Event Catering Standards Director',
            'Convention Center Food Policy Manager',
            'Theme Park Restaurant Operations Director',
            'Performance Venue Catering Manager',
            'Festival Food Vendor Coordinator',
            'Exhibition Center Menu Planner',
            'Arena Sustainability Program Director',
            'Event Space Food Innovation Manager',
            'Venue Dietary Requirements Coordinator',
            'Large-Scale Food Safety Director',
            'Sports Complex Nutrition Manager',
            'Entertainment Venue Supply Chain Director',

            # Pharmaceutical & Medical Materials
            'Animal-Derived Medicine Researcher',
            'Medical Device Industry Materials Director',
            'Pharmaceutical Testing Standards Manager',
            'Medical Supply Chain Director',
            'Vaccine Production Materials Manager',
            'Biomedical Research Ethics Officer',
            'Medical Materials Sourcing Director',
            'Pharmaceutical Alternatives Researcher',
            'Healthcare Products Standards Manager',
            'Medical Manufacturing Compliance Director',
            'Drug Development Ethics Officer',
            'Medical Materials Innovation Director',
            'Healthcare Sustainability Manager',
            'Pharmaceutical Supply Chain Transformer',
            'Medical Research Standards Director',
            'Healthcare Materials Transition Manager',

            # Labor Union Leadership
            'Meat Industry Union Leader',
            'Food Workers Union Policy Director',
            'Agricultural Workers Union Representative',
            'Slaughterhouse Workers Union Official',
            'Farm Workers Rights Advocate',
            'Food Industry Labor Relations Manager',
            'Agricultural Union Negotiation Director',
            'Worker Safety Standards Coordinator',
            'Labor Rights Education Director',
            'Union-Management Liaison Officer',
            'Worker Transition Program Manager',
            'Labor Policy Development Director',
            'Worker Advocacy Campaign Manager',
            'Union Membership Outreach Director',
            'Labor Standards Compliance Officer',
            'Worker Support Program Coordinator',

            # Insurance & Risk Assessment
            'Agricultural Insurance Underwriter',
            'Livestock Industry Risk Assessor',
            'Food Production Liability Manager',
            'Animal Agriculture Insurance Policy Director',
            'Food Safety Insurance Specialist',
            'Farm Practice Risk Analyst',
            'Agricultural Policy Underwriting Manager',
            'Food Industry Insurance Developer',
            'Livestock Operation Risk Manager',
            'Production Facility Safety Assessor',
            'Industry Transition Risk Specialist',
            'Agricultural Claims Investigation Director',
            'Food System Risk Modeling Expert',
            'Insurance Policy Innovation Manager',
            'Industry Safety Standards Director',
            'Risk Mitigation Program Coordinator',

            # Trade Association Leadership
            'Meat Industry Association President',
            'Plant-Based Foods Association Director',
            'Agricultural Trade Group Leader',
            'Food Industry Coalition Manager',
            'Industry Standards Board Chair',
            'Trade Association Policy Director',
            'Industry Relations Coordinator',
            'Member Services Program Manager',
            'Trade Group Advocacy Director',
            'Industry Development Officer',
            'Association Communications Manager',
            'Stakeholder Relations Director',
            'Industry Research Program Manager',
            'Trade Organization Strategy Director',
            'Association Governance Officer',
            'Industry Partnership Coordinator',

            # Certification & Auditing
            'Animal Welfare Certification Director',
            'Food Safety Audit Manager',
            'Industry Standards Verification Officer',
            'Supply Chain Certification Specialist',
            'Production Practice Auditor',
            'Ethical Sourcing Certification Manager',
            'Quality Assurance Program Director',
            'Standards Compliance Auditor',
            'Certification Program Developer',
            'Industry Practice Inspector',
            'Audit Protocol Development Manager',
            'Verification Systems Director',
            'Standards Implementation Officer',
            'Certification Training Manager',
            'Audit Methodology Developer',
            'Compliance Assessment Director',

            # Government Relations
            'Agricultural Policy Liaison',
            'Food Industry Government Affairs Director',
            'Legislative Relations Manager',
            'Regulatory Affairs Coordinator',
            'Political Strategy Advisor',
            'Government Outreach Director',
            'Policy Development Manager',
            'Legislative Advocacy Coordinator',
            'Regulatory Compliance Liaison',
            'Government Programs Director',
            'Policy Implementation Manager',
            'Agency Relations Coordinator',
            'Legislative Impact Analyst',
            'Government Affairs Strategist',
            'Regulatory Change Manager',
            'Policy Engagement Director',

            # International Trade Organizations
            'Agricultural Trade Representative',
            'Food Export Association Director',
            'International Standards Coordinator',
            'Trade Policy Development Manager',
            'Global Market Access Director',
            'International Relations Manager',
            'Trade Agreement Negotiator',
            'Export Compliance Director',
            'International Market Developer',
            'Trade Promotion Coordinator',
            'Global Standards Manager',
            'International Trade Liaison',
            'Market Development Director',
            'Trade Policy Analyst',
            'International Compliance Manager',
            'Global Partnership Director'

            # Economic
            'Investment Director for Traditional Livestock Operations',
            'Analyst Criticizing Plant-Based Market Claims',
            'Fund Manager Specializing in Animal Agriculture Growth',
            'Economic Researcher Challenging Vegan Market Viability',
            'Business Consultant for Industrial Farming Operations',
            'Venture Capitalist Funding Plant-Based Innovations',
            'Corporate Strategist Focused on ESG Investments',
            'Investor in Industrial Livestock Operations',
            'Financial Analyst Assessing Ethical Consumer Trends',
            'Economist Focused on Global Food Security',
            'Commodity Trader Specializing in Animal Products',
            'Agricultural Futures Analyst',
            'Impact Investment Fund Manager',
            'Food Industry Merger & Acquisition Specialist',
            'Sustainable Agriculture Investment Advisor',
            'Agricultural Technology Investment Director',
            'Food Systems Economic Development Officer',
            'Alternative Protein Market Analyst',
            'Agricultural Insurance Underwriter',
            'Food Innovation Grant Manager',
            'Agricultural Real Estate Investment Trust Manager',

            # Societal & Individuals
            'Anti-Vegan Movement Leader',
            'Traditional Foods Preservation Activist',
            'Critic of Plant-Based Health Claims',
            'Cultural Heritage Defender Against Veganization',
            'Advocate for Maintaining Traditional Food Systems',
            'Social Media Influencer Against Vegan Lifestyle',
            'Community Leader Opposing Vegan School Meals',
            'Content Creator Debunking Vegan Arguments',
            'Local Food Security Advocate Supporting Animal Agriculture',
            'Protest Organizer Against Vegan Business Expansion',
            'Community Organizer Promoting Plant-Based Initiatives',
            'Neighborhood Leader Supporting Diverse Food Choices',
            'Cultural Icon Celebrating Traditional Meat-Based Diets',
            'Social Media Advocate for Plant-Based Lifestyles',
            'Content Creator Exploring Both Vegan and Non-Vegan Recipes',
            'Public Speaker Highlighting Ranching as a Cultural Heritage',
            'Food Justice Activist',
            'Indigenous Food Systems Preservationist',
            'Urban Food Desert Solutions Advocate',
            'Food Waste Reduction Champion',
            'Community Garden Coordinator',
            'Food Literacy Educator',
            'Cultural Food Preservation Specialist',
            'Food Access Rights Advocate',
            'Local Food Movement Leader',
            'Food Sovereignty Activist',

            # Influential Individuals
            'Celebrity Donating to Vegan Causes',
            'Public Figure Promoting Food Freedom',
            'Chef Endorsing Traditional Farm-to-Table Dining',
            'Entrepreneur Running a Plant-Based Business',
            'Entrepreneur Running a Heritage Meat Business',
            'Musician Writing Songs about Ethical Food Choices',
            'Actor Promoting Environmental Veganism',
            'Professional Athlete Advocating Plant-Based Performance',
            'Food Network Personality',
            'Culinary Innovation Ambassador',
            'Sustainable Living Expert',
            'Environmental Documentary Producer',
            'Food Systems Author',
            'Podcast Host Focusing on Food Ethics',
            'Sustainable Agriculture Influencer',
            'Food Technology Thought Leader',

            # Religious
            'Religious Leader Advocating Compassionate Plant-Based Practices',
            'Cleric Teaching Moderation in Dietary Habits',
            'Religious Figure Endorsing Ethical Animal Husbandry',
            'Faith-Based Sustainability Advisor',
            'Spiritual Guide Promoting Vegan as Sacred Practice',
            'Monastic Leader Incorporating Locally Sourced Meats',
            'Interfaith Food Ethics Coordinator',
            'Religious Environmental Stewardship Director',
            'Sacred Foods Preservation Specialist',
            'Faith-Based Community Garden Manager',
            'Religious Dietary Law Expert',
            'Spiritual Ecology Teacher',
            'Religious Food Justice Advocate',
            'Faith-Based Sustainable Agriculture Advisor',
            'Sacred Land Management Consultant',
            'Religious Food Systems Scholar',

            # Educational
            'Teacher Introducing Plant-Based Concepts in Curriculums',
            'Principal Supporting Diverse Cafeteria Options',
            'University Researcher Focusing on Livestock Sustainability',
            'Educator on Environmental Impacts of Meat Production',
            'Dean of Agriculture Promoting Regenerative Farming',
            'College Professor Examining Cultural Food Systems',
            'Agricultural Extension Officer',
            'Food Science Program Director',
            'Sustainable Agriculture Curriculum Developer',
            'School Garden Program Coordinator',
            'Food Systems Research Fellow',
            'Agricultural Education Technology Specialist',
            'Youth Farming Program Director',
            'Food History Professor',
            'Nutrition Education Coordinator',
            'Agricultural Vocational Training Director',

            # News Media
            'Journalist Writing About Veganisms Benefits',
            'Neutral Reporter Covering Food Industry Trends',
            'Columnist Defending Traditional Farming Practices',
            'Media Producer Focused on Ethical Eating Documentaries',
            'Editor Publishing Both Plant-Based and Meat-Based Recipes',
            'Broadcast Host Exploring the Ethics of Meat',
            'Food Industry Investigative Reporter',
            'Agricultural Policy Correspondent',
            'Food Innovation Tech Writer',
            'Sustainable Agriculture Documentary Producer',
            'Food Culture Podcast Host',
            'Agricultural Trade Publication Editor',
            'Food Science Communications Director',
            'Rural Affairs Correspondent',
            'Food Systems Video Journalist',
            'Agricultural Technology News Editor',

            # Health
            'Doctor Criticizing Vegan Diets',
            'Nutritionist Promoting Animal-Based Proteins',
            'Research Scientist Studying Vegan Diet Deficiencies',
            'Sports Medicine Doctor Opposing Plant-Based Athletics',
            'Clinical Nutritionist Specializing in Animal-Based Healing',
            'Health Researcher Documenting Vegan Diet Risks',
            'Pediatrician Advocating Against Child Veganism',
            'Public Health Expert Warning About Vegan Deficiencies',
            'Medical Professional Promoting Carnivore Diet',
            'Healthcare Provider Specializing in Ex-Vegan Recovery',
            'Nutritionist Advocating Whole Food Plant-Based Diets',
            'Dietitian Supporting Balanced Eating Choices',
            'Doctor Advising on High-Protein Animal Diets',
            'Fitness Guru Promoting Vegan Performance',
            'Therapist Helping with Vegan Transition Challenges',
            'Medical Practitioner Treating Nutritional Deficiencies in Vegans',
            'Sports Medicine Specialist in Plant-Based Nutrition',
            'Pediatric Nutritionist',
            'Clinical Research Dietitian',
            'Eating Disorder Treatment Specialist',
            'Public Health Nutrition Director',
            'Corporate Wellness Program Manager',
            'Metabolic Health Specialist',
            'Geriatric Nutrition Consultant',
            'Performance Nutrition Coach',
            'Integrative Medicine Practitioner',

            # Sports
            'Athlete Endorsing Plant-Based Nutrition for Performance',
            'Neutral Sports Coach Open to All Diet Choices',
            'Trainer Promoting Meat-Based Diets for Muscle Building',
            'Owner of a Vegan Fitness Studio',
            'Gym Franchise Manager Offering Mixed Meal Plans',
            'Athletic Brand Ambassador for Traditional Protein Sources',
            'Olympic Training Nutritionist',
            'Professional Sports Team Dietitian',
            'Combat Sports Nutrition Coach',
            'Endurance Athletics Specialist',
            'Sports Recovery Expert',
            'Youth Sports Nutrition Advisor',
            'Athletic Performance Researcher',
            'Sports Medicine Chef',
            'Team Nutrition Coordinator',
            'Sports Supplement Developer',

            # Transportation
            'Designer Creating Animal-Free Automotive Interiors',
            'Logistics Manager Offering Vegan Meal Options in Transit',
            'Car Manufacturer Using Traditional Leather Interiors',
            'Aviation Executive Championing Plant-Based Catering',
            'Neutral Shipping Executive Focusing on Cargo Versatility',
            'Automaker Supporting Classic Materials in Design',
            'Sustainable Transport Materials Engineer',
            'Railway Food Service Director',
            'Cruise Line Culinary Manager',
            'Electric Vehicle Interior Designer',
            'Alternative Materials Research Director',
            'Transport Sustainability Consultant',
            'Food Transport Logistics Specialist',
            'Aviation Menu Planning Director',
            'Maritime Catering Manager',
            'Transport Industry Ethics Advisor',

            # Hospitality
            'Vegan Chef Designing Exclusive Plant-Based Menus',
            'Restaurateur Balancing Meat and Vegan Offerings',
            'Hotelier Emphasizing Local Meat Dishes in Menus',
            'Caterer Specializing in Vegan Events',
            'Neutral Event Planner Accommodating All Dietary Needs',
            'Baker Focusing on Traditional Butter-Based Pastries',
            'Food Festival Coordinator',
            'Corporate Dining Services Manager',
            'Wedding Menu Specialist',
            'Hotel Procurement Manager',
            'Restaurant Sustainability Consultant',
            'Culinary Education Director',
            'Food Innovation Chef',
            'Hospital Food Service Director',
            'School Cafeteria Manager',
            'Airlines Catering Executive',

            # Retail
            'Supermarket Owner Expanding Vegan Product Lines',
            'Retail Manager Balancing Consumer Preferences',
            'Shopkeeper Specializing in Local Animal Products',
            'E-Commerce Vendor Selling Plant-Based Goods',
            'Store Buyer Offering Both Vegan and Non-Vegan Selections',
            'Entrepreneur Creating Demand for Dairy-Based Snacks',
            'Specialty Food Store Owner',
            'Farmers Market Coordinator',
            'Food Co-op Manager',
            'Online Grocery Platform Developer',
            'Retail Analytics Specialist',
            'Food Product Category Manager',
            'International Food Import Specialist',
            'Local Food Distribution Coordinator',
            'Sustainable Packaging Consultant',
            'Food Waste Reduction Manager',

            # Technology
            'Engineer Developing Next-Gen Plant-Based Meats',
            'Technologist Innovating in Food Science',
            'Startup Founder Focused on Cultured Meat',
            'Tech Investor Exploring Diverse Food Tech Markets',
            'Lab Technician Creating Hybrid Meat-Plant Foods',
            'Food Scientist Enhancing Traditional Animal Proteins',
            'Alternative Protein Research Director',
            'Food Technology Patent Attorney',
            'Cellular Agriculture Specialist',
            'Food Processing Automation Engineer',
            'Precision Fermentation Researcher',
            'Smart Agriculture Systems Developer',
            'Food Safety Technology Manager',
            'Bioprocess Engineering Director',
            'Food Packaging Innovation Specialist',
            'Agricultural Robotics Engineer',

            # Environmental
            'Climate Activist Promoting Veganism as a Solution',
            'Conservationist Balancing Livelihoods and Ecosystems',
            'Environmentalist Supporting Ethical Ranching Practices',
            'Researcher Highlighting Carbon Savings from Plant Diets',
            'Neutral Ecologist Evaluating Farming Practices',
            'Advocate for Grass-Fed and Rotational Grazing Systems',
            'Soil Health Specialist',
            'Water Conservation Expert',
            'Biodiversity Protection Coordinator',
            'Sustainable Agriculture Consultant',
            'Carbon Footprint Analyst',
            'Environmental Impact Assessor',
            'Regenerative Agriculture Specialist',
            'Wildlife Corridor Manager',
            'Ecosystem Services Researcher',
            'Agricultural Waste Management Expert',

            # Real Estate
            'Developer Investing in Plant-Based Industries',
            'Landowner Open to Sustainable Food Enterprises',
            'Property Developer Partnering with Traditional Farms',
            'Urban Planner Designing Vegan-Friendly Spaces',
            'Architect Integrating Both Ethical and Traditional Spaces',
            'Land Steward Promoting Cattle Grazing Practices',
            'Agricultural Land Trust Manager',
            'Vertical Farming Facility Developer',
            'Food Hub Real Estate Specialist',
            'Urban Agriculture Space Planner',
            'Sustainable Development Consultant',
            'Agricultural Zoning Expert',
            'Food District Design Manager',
            'Green Building Certification Specialist',
            'Rural Development Project Manager',
            'Food Processing Facility Designer',

            # Manufacturing
            'Industrial Animal Processing Plant Manager',
            'Traditional Leather Goods Manufacturer',
            'Animal Testing Equipment Producer',
            'Factory Farm Technology Developer',
            'Slaughterhouse Equipment Designer',
            'Animal Transport Vehicle Manufacturer',
            'Industrial Livestock Feed Producer',
            'Traditional Dairy Equipment Manufacturer',
            'Animal Containment Systems Engineer',
            'Livestock Processing Automation Expert',
            'Producer Using Plant-Based Textiles',
            'Factory Manager Shifting to Ethical Supply Chains',
            'Manufacturer Supporting Animal-Derived Materials',
            'Startup Innovating Vegan Fabrics',
            'Neutral Manufacturer Meeting Market Demands',
            'Legacy Factory Preserving Animal-Product-Based Goods',
            'Sustainable Materials Engineer',
            'Clean Technology Implementation Manager',
            'Alternative Leather Developer',
            'Bio-Based Materials Researcher',
            'Circular Economy Specialist',
            'Zero-Waste Manufacturing Consultant',
            'Quality Control Manager for Vegan Products',
            'Supply Chain Sustainability Director',
            'Eco-Friendly Packaging Designer',
            'Manufacturing Process Innovation Expert',

            # Tourism
            'Travel Agent Specializing in Vegan Experiences',
            'Hospitality Consultant Catering to All Preferences',
            'Tour Operator Celebrating Farm-to-Table Meats',
            'Resort Manager Offering Plant-Based Packages',
            'Travel Guide Promoting Traditional Local Foods',
            'Adventurer Leading Eco-Tours with Ethical Choices',
            'Food Tourism Development Manager',
            'Culinary Tour Guide',
            'Agritourism Coordinator',
            'Sustainable Tourism Consultant',
            'Food Festival Event Planner',
            'Cultural Food Experience Designer',
            'Farm Stay Program Manager',
            'Wine Tourism Specialist',
            'Eco-Lodge Owner',
            'Food Heritage Tour Developer',

            # Entertainment Media
            'Director Creating Documentaries About Veganism',
            'Producer Developing Balanced Food Industry Narratives',
            'Filmmaker Highlighting Farming Traditions',
            'Game Designer Featuring Ethical Plant-Based Stories',
            'Screenwriter Exploring Veganism and Cultural Foods',
            'Documentary Maker Defending Livestock Communities',
            'Food Show Producer',
            'Cooking Competition Show Creator',
            'Agricultural Reality Show Developer',
            'Food Culture Series Director',
            'Farm Life Documentary Producer',
            'Sustainable Living Show Host',
            'Food Innovation Program Creator',
            'Agricultural Heritage Documentarian',
            'Food Science Show Writer',
            'Culinary Travel Show Producer',

            # Land Animal Farmers
            'Dairy Farmer Transitioning to Plant-Based Farming',
            'Mixed Farmer Balancing Crops and Livestock',
            'Beef Producer Advocating for Ethical Meat Practices',
            'Pastoralist Exploring Crop Diversification',
            'Heritage Farmer Maintaining Traditional Livestock Practices',
            'Rancher Emphasizing Sustainable Grazing',
            'Regenerative Livestock Farmer',
            'Integrated Crop-Livestock Manager',
            'Organic Dairy Operation Owner',
            'Free-Range Poultry Farmer',
            'Permaculture Farm Manager',
            'Rotational Grazing Specialist',
            'Small-Scale Family Farm Owner',
            'Urban Animal Husbandry Practitioner',
            'Heritage Breed Conservator',
            'Sustainable Sheep Farmer',

            # Aquaculture
            'Seaweed Farmer Promoting Plant-Based Alternatives',
            'Fisherman Balancing Harvest and Conservation',
            'Shrimp Farmer Advocating Sustainable Practices',
            'Aquaculture Specialist Transitioning to Vegan Solutions',
            'Neutral Mariculture Expert',
            'Fish Farmer Optimizing Animal Protein Yields',
            'Algae Production Manager',
            'Sustainable Aquaponics Director',
            'Marine Plant Cultivation Specialist',
            'Recirculating Aquaculture Systems Expert',
            'Integrated Multi-Trophic Aquaculture Manager',
            'Aquatic Feed Development Researcher',
            'Fish Health Management Specialist',
            'Shellfish Farming Coordinator',
            'Ocean Farming Innovation Director',
            'Aquaculture Certification Manager',

            # Wild Fisheries
            'Advocate for Ocean-Friendly Vegan Solutions',
            'Sustainable Fisheries Researcher',
            'Recreational Fisher Protecting Local Traditions',
            'Commercial Fisher Exploring Non-Meat Revenue Streams',
            'Marine Ecologist Supporting Both Conservation and Industry',
            'Advocate for Cultural Fisheries Practices',
            'Small-Scale Fisheries Manager',
            'Indigenous Fishing Rights Advocate',
            'Marine Protected Areas Coordinator',
            'Fishing Community Development Officer',
            'Fisheries Policy Analyst',
            'Seafood Sustainability Auditor',
            'Traditional Fishing Methods Preservationist',
            'Fish Stock Assessment Specialist',
            'Marine Resource Management Director',
            'Artisanal Fishing Advocate',

            # Animal Testing Labs
            'Researcher Innovating in Vegan Testing Methods',
            'Lab Technician Evaluating Alternative Approaches',
            'Scientist Defending Legacy Testing Standards',
            'Alternative Testing Methods Developer',
            'In Vitro Research Specialist',
            'Computational Toxicology Expert',
            'Animal-Free Testing Advocate',
            'Regulatory Compliance Officer',
            'Ethics Committee Member',
            'Research Methods Innovation Director',
            'Validation Studies Coordinator',
            'Safety Assessment Specialist',
            'Non-Animal Methods Trainer',
            'Testing Protocol Developer',
            'Alternatives Assessment Manager',
            'Research Ethics Consultant',

            # Fur Farms
            'Entrepreneur Transitioning from Fur to Vegan Alternatives',
            'Ethical Consultant for Former Fur Farmers',
            'Legacy Furrier Preserving Traditional Practices',
            'Sustainable Textiles Developer',
            'Alternative Materials Researcher',
            'Farm Conversion Specialist',
            'Animal Welfare Auditor',
            'Rehabilitation Program Coordinator',
            'Traditional Skills Preservationist',
            'Economic Transition Advisor',
            'Wildlife Conservation Expert',
            'Alternative Income Development Specialist',
            'Heritage Crafts Educator',
            'Sustainable Fashion Consultant',
            'Animal Sanctuary Developer',
            'Business Transformation Coach',

            # Fashion Industry
            'Designer Embracing Vegan Materials',
            'Neutral Retail Buyer Offering Mixed Options',
            'Couturier Specializing in Animal-Based Luxury',
            'Sustainable Fashion Innovation Director',
            'Circular Fashion Economy Expert',
            'Bio-Based Materials Developer',
            'Ethical Fashion Brand Manager',
            'Fashion Technology Researcher',
            'Traditional Craftsmanship Preservationist',
            'Supply Chain Transparency Director',
            'Eco-Textile Innovation Specialist',
            'Fashion Sustainability Consultant',
            'Alternative Materials Sourcing Manager',
            'Heritage Fashion Curator',
            'Ethical Production Coordinator',
            'Fashion Future Trends Analyst',

            # Companion Animal Industry
            'Pet Food Innovator Developing Vegan Products',
            'Shelter Operator Promoting Animal Adoption',
            'Pet Store Owner Focusing on Mixed Offerings',
            'Alternative Pet Nutrition Specialist',
            'Companion Animal Welfare Director',
            'Pet Food Safety Researcher',
            'Animal Behavior Consultant',
            'Rescue Organization Coordinator',
            'Pet Industry Ethics Advisor',
            'Sustainable Pet Products Developer',
            'Animal Health Technology Specialist',
            'Pet Care Education Director',
            'Species-Specific Diet Expert',
            'Animal Rehabilitation Specialist',
            'Pet Industry Standards Developer',
            'Animal-Human Bond Researcher',

            # Prisons
            'Prison Reformer Supporting Plant-Based Menus',
            'Facility Manager Balancing Dietary Needs',
            'Warden Advocating for Traditional Meal Options',
            'Correctional Food Service Director',
            'Inmate Nutrition Program Manager',
            'Prison Garden Program Coordinator',
            'Food Security Specialist',
            'Dietary Accommodation Coordinator',
            'Institutional Menu Planner',
            'Food Safety Compliance Officer',
            'Prison Kitchen Training Manager',
            'Sustainable Operations Director',
            'Cultural Food Programs Coordinator',
            'Special Diets Administrator',
            'Food Cost Management Specialist',
            'Meal Service Quality Controller',

            # "Pest" Control
            'Relocation Expert Specializing in Humane Practices',
            'Neutral Pest Control Operator',
            'Exterminator Using Conventional Methods',
            'Integrated Pest Management Specialist',
            'Wildlife Conflict Resolution Expert',
            'Humane Wildlife Control Manager',
            'Natural Deterrent Systems Developer',
            'Urban Wildlife Coexistence Coordinator',
            'Ecological Balance Consultant',
            'Prevention Systems Designer',
            'Habitat Modification Specialist',
            'Non-Lethal Solutions Expert',
            'Biological Control Researcher',
            'Urban Ecology Management Director',
            'Animal Behavior Modification Specialist',
            'Sustainable Pest Management Consultant',

            # Hunting
            'Conservationist Advocating Reduced Hunting',
            'Gamekeeper Balancing Wildlife and Hunting Interests',
            'Hunter Promoting Ethical Harvesting',
            'Wildlife Population Manager',
            'Habitat Conservation Specialist',
            'Traditional Hunting Methods Educator',
            'Subsistence Hunting Advocate',
            'Wildlife Research Coordinator',
            'Conservation Policy Advisor',
            'Hunting Safety Instructor',
            'Land Use Management Expert',
            'Species Protection Officer',
            'Hunt Club Environmental Director',
            'Wildlife Corridor Planner',
            'Hunting Ethics Educator',
            'Ecosystem Management Specialist',

            # Cultured Food Companies
            'Innovator Leading Cultured Meat Development',
            'Executive Marketing Lab-Based Protein Alternatives',
            'Skeptic of Lab-Grown Foods Supporting Traditional Meats',
            'Cellular Agriculture Researcher',
            'Bioprocess Development Manager',
            'Food Safety Compliance Director',
            'Scale-Up Operations Specialist',
            'Product Development Scientist',
            'Alternative Protein Market Analyst',
            'Regulatory Affairs Manager',
            'Quality Assurance Director',
            'Consumer Research Specialist',
            'Tissue Engineering Expert',
            'Production Technology Manager',
            'Innovation Research Director',
            'Food Technology Strategist',

            # Plant-Based Food Companies
            'Marketing Manager for Vegan Products',
            'Neutral Operations Specialist in Food Industry',
            'Food Company CEO Expanding Dairy-Based Offerings',
            'Plant Protein Development Scientist',
            'Alternative Dairy Innovation Director',
            'Product Formulation Specialist',
            'Sustainable Sourcing Manager',
            'Food Technology Research Lead',
            'Consumer Insight Analyst',
            'Quality Control Supervisor',
            'Plant-Based Manufacturing Director',
            'Ingredient Innovation Specialist',
            'Nutrition Research Coordinator',
            'Brand Development Manager',
            'Supply Chain Sustainability Lead',
            'Food Safety Compliance Manager',

            # Animal Rights/Vegan Organisations
            'Volunteer Championing Animal-Free Lifestyles',
            'Staff Member Bridging Vegan and Non-Vegan Conversations',
            'Leader Advocating for Ethical Meat Alternatives',
            'Campaign Strategy Director',
            'Corporate Outreach Coordinator',
            'Education Program Manager',
            'Policy Research Specialist',
            'Community Engagement Director',
            'Legal Rights Advocate',
            'Grant Writing Specialist',
            'Social Media Campaign Manager',
            'Volunteer Coordination Director',
            'Event Planning Manager',
            'Public Relations Specialist',
            'Development Director',
            'Youth Education Coordinator',

            # Core Campaign Roles
            'Digital Campaign Director',
            'Policy Research Director',
            'Grassroots Organizing Director',
            'Digital Content Strategist',
            'Development Director',
            'Corporate Engagement Director',
            'Communications Director',
            'Education Program Director',
            'Welfare Campaign Director',
            'Rights Campaign Director',

            # Independent Advocacy
            'Independent Strategy Consultant',
            'Grassroots Activist',
            'Intersectional Research Director',
            'Impact Investment Director',
            'Corporate Pressure Campaign Director',
            'Animal Welfare Attorney',
            'Animal Rights Attorney',

            # Care & Support
            'Sanctuary Operations Director',
            'Independent Rescuer',
            'Research Communication Director',
            'Creative Content Director',
            'Animal Welfare Science Director',
            'Animal Rights Theory Director',
            'AI Ethics Lead',
            'Data Analytics Director',

            # Intelligence & Investigation
            'OSINT Research Director',
            'Digital Security Director',
            'Information Access Specialist',
            'Innovation Director',
            'Network Intelligence Director',
            'Digital Evidence Specialist',
            'Remote Sensing Specialist',
            'Supply Chain Analyst',

            # Technical Operations
            'Aerial Documentation Director',
            'Immersive Technology Director',
            'Environmental Sound Analyst',
            'Environmental Impact Specialist',
            'Information Systems Director',
            'Engagement Strategy Director',
            'Environmental Testing Specialist',
            'Industry Economics Specialist',

            # Strategic Communications
            'Inclusive Communications Director',
            'Knowledge Systems Director',
            'Cognitive Science Director',
            'Cultural Analysis Director',
            'Behavioral Change Director',
            'Neural Impact Director',
            'Network Dynamics Director',
            'Neural Interface Director',

            # Advanced Strategy
            'Viral Dynamics Director',
            'Subconscious Change Director',
            'Animal Communication Director',
            'Evolutionary Dynamics Director',
            'Quantum Biology Director',
            'Social Contagion Director',
            'Nonviolent Strategy Director',
            'Economic Pressure Director',

            # Intelligence & Analysis
            'Industry Intelligence Director',
            'Reputation Impact Director',
            'Strategic Communications Director',
            'Strategic Research Director',
            'Supply Chain Strategy Director',
            'Regulatory Strategy Director',
            'Digital Strategy Director',
            'Market Research Director',

            # Movement Coordination
            'Distributed Action Director',
            'Asymmetric Strategy Director',
            'Security Operations Director',
            'Digital Evidence Director',
            'Digital Infrastructure Director',
            'Digital Intelligence Director',
            'Threat Assessment Director',
            'Information Access Director',

            # Technical Investigation
            'Blockchain Intelligence Director',
            'Counter-Surveillance Director',
            'Geospatial Intelligence Director',
            'Digital Presence Director',
            'Communications Security Director',
            'Worker Rights Strategist',
            'Strategic Research Director',
            'Crisis Strategy Director',

            # International & Systems
            'International Advocacy Director',
            'Network Analysis Director',
            'Industry Investigation Director',
            'Belief Systems Director',
            'Strategic Intelligence Director',
            'Rights Campaign Director',
            'Strategic Negotiation Director',
            'Campaign Strategy Director',

            # Psychology & Behavior
            'Behavioral Strategy Director',
            'Investigation Strategy Director',
            'Message Analysis Director',
            'Mass Behavior Director',
            'Future Scenarios Director',
            'Biotech Strategy Director',
            'AR Experience Director',
            'Quantum Systems Director',

            # Advanced Operations
            'Memetic Operations Director',
            'Cognitive Strategy Director',
            'Systems Disruption Director',
            'Cultural Change Director',
            'Algorithmic Strategy Director',
            'Narrative Strategy Director',
            'Automation Strategy Director',
            'Quantum Social Dynamics Director',

            # Technical Development
            'Advanced Materials Director',
            'Neural Systems Director',
            'Collective Systems Director',
            'Social Physics Director',
            'Decentralized Systems Director',
            'Natural Systems Director',
            'Emergent Phenomena Director',
            'Social Computing Director',

            # Systems & Influence
            'Persuasion Systems Director',
            'Social Encryption Director',
            'Reality Design Director',
            'Morphic Resonance Director',
            'Strategic Dynamics Director',
            'Quantum Systems Director',
            'Biofield Research Director',
            'Consciousness Systems Director',

            # Evolution & Change
            'Social Evolution Director',
            'Metamodern Systems Director',
            'Reality Design Director',
            'Time Perception Director',
            'Memetic Evolution Director',
            'Coherence Systems Director',
            'Emergence Systems Director',
            'Economic Transition Director',

            # Systems Transformation
            'Food Systems Director',
            'Standards Development Director',
            'Procurement Strategy Director',
            'Health Policy Director',
            'Trade Policy Director',
            'Risk Assessment Director',
            'Urban Systems Director',
            'Packaging Systems Director',

            # Rights & Reform
            'Safety Systems Director',
            'Prison Rights Director',
            'Wildlife Policy Director',
            'Interfaith Strategy Director',
            'Companion Animal Rights Director',
            'Creative Disruption Director',
            'Digital Action Director',
            'Cultural Events Director',

            # Support & Standards
            'Advocate Wellbeing Director',
            'Standards Director',
            'Arts Strategy Director',
            'Strategic Pressure Director',
            'Digital Protest Director',
            'Fashion Innovation Director',
            'Ethical Tourism Director',
            'Hunger Strike Safety Director',

            # Industry Transformation
            'Transport Innovation Director',
            'Property Strategy Director',
            'Aquaculture Transformation Director',
            'Media Transformation Director',
            'Occupation Strategy Director',
            'Sports Transformation Director',
            'Manufacturing Innovation Director',
            'Healthcare Transformation Director',

            # Education & Learning
            'Education Innovation Director',
            'Fisheries Transformation Director',
            'Direct Action Safety Director',
            'Labeling Standards Director',
            'Political Strategy Director',
            'Economic Strategy Director',
            'Strike Strategy Director',
            'Impact Assessment Director',

            # Technology & Tools
            'Digital Tools Director',
            'Communications Director',
            'Visual Impact Director',
            'Innovation Events Director',
            'Learning Experience Director',
            'Technical Implementation Director',
            'Digital Solutions Director',
            'Systems Integration Director'

            # Individual Advocates
            'Blogger Promoting Plant-Based Awareness',
            'Lifestyle Influencer Exploring All Diet Choices',
            'Food Critic Defending Traditional Cuisine',
            'Social Media Content Creator',
            'Podcast Host Discussing Food Ethics',
            'Community Workshop Facilitator',
            'Online Course Developer',
            'Public Speaker on Food Issues',
            'Newsletter Publisher',
            'Documentary Filmmaker',
            'Recipe Developer and Tester',
            'Health Coach',
            'Food Photography Specialist',
            'Book Author',
            'YouTube Channel Manager',
            'Personal Brand Consultant',

            # Animal Advocates
            'Volunteer for an Animal Advocacy Organisation',
            'Donor to an Animal Advocacy Organisation',
            'Staff Member of an Animal Advocacy Organisation',
            'Researcher Studying Animal Advocacy Issues',
            'Independent Animal Advocate',
            'Animal Lawyer or Legal Advocate',
            'Animal Carer or Rescuer',
            'Vegan Influencer, Blogger or Content Creator',
            'Owner of a Vegan or Cruelty-Free Company',
            'Staff Member of a Vegan or Cruelty-Free Company',
            'Investor in a Vegan or Cruelty-Free Company',

            # Media and Information Warfare
            'specialist in mass media system manipulation',
            'expert in industry propaganda disruption',
            'coordinator of leaked document campaigns',
            'specialist in critical information exposure',
            'expert in narrative control operations',
            'coordinator of systematic truth revelation',
            'expert in industry secret mass distribution',
            'specialist in media infrastructure disruption',
            'coordinator of global information campaigns',
            'expert in social media system manipulation',

            # Research and Development
            'designer of custom liberation equipment',
            'expert in resistance technology development',
            'specialist in security system analysis',
            'coordinator of technical innovation programs',
            'expert in resistance tool fabrication',
            'specialist in equipment adaptation',
            'developer of specialized breach tools',
            'expert in countermeasure development',
            'specialist in tactical equipment design',
            'coordinator of technical advancement programs',

            # International Operations
            'coordinator of cross-border resistance operations',
            'expert in international resource movement',
            'specialist in global network coordination',
            'expert in multi-national campaign planning',
            'coordinator of international intelligence sharing',
            'specialist in global resistance alignment',
            'expert in international pressure campaigns',
            'coordinator of worldwide resistance networks',
            'specialist in international sanctuary networks',
            'expert in global movement coordination',

            # Industry Worker Subversion
            'specialist in employee resistance cultivation',
            'expert in worker solidarity building',
            'coordinator of internal resistance networks',
            'specialist in employee loyalty disruption',
            'expert in worker defection operations',
            'coordinator of mass walk-out campaigns',
            'specialist in workforce disruption tactics',
            'expert in employee radicalization',
            'coordinator of insider resistance cells',
            'specialist in worker network development',

            # Legal System Navigation
            'expert in resistance legal protection',
            'specialist in evidence elimination',
            'coordinator of legal defense networks',
            'expert in operational security protocols',
            'specialist in movement protection strategies',
            'coordinator of rapid response legal teams',
            'expert in digital trace elimination',
            'specialist in resistance documentation security',
            'coordinator of movement legal infrastructure',
            'expert in security protocol development'

            # Core militant roles
            'commander of an animal liberation resistance army',
            'leader of an armed animal liberation resistance cell',
            'coordinator of militant direct action networks',
            'commander of a liberation tactical unit',
            'leader of a militant sanctuary defense force',
            'commander of industrial sabotage units',
            'member of the Animal Liberation Front',
            'member of the Earth Liberation Front',
            'member of Direct Action Everywhere',
            'member of the Green Mountain Anarchist Collective',
            'coordinator of revolutionary liberation brigades',
            'organizer of armed resistance operations',

            # Armed resistance specialists
            'specialist in armed facility liberation',
            'expert in tactical resistance operations',
            'specialist in defended sanctuary establishment',
            'expert in militant blockade tactics',
            'coordinator of armed transport intervention',
            'specialist in facility occupation defense',
            'expert in armed resistance strategies',
            'coordinator of tactical response teams',
            'specialist in defensive perimeter control',
            'expert in resistance force protection',
            'specialist in forceful facility shutdown',
            'expert in rapid entry and exit strategies',
            'coordinator of synchronized multi-site actions',

            # Advanced Technology and Cybersecurity
            'expert in industrial control system exploitation',
            'specialist in SCADA system disruption',
            'coordinator of systematic network breaches',
            'expert in supply chain software manipulation',
            'specialist in facility automation takeover',
            'coordinator of mass system failure cascades',
            'expert in biometric security circumvention',
            'specialist in access control system breaches',
            'coordinator of critical infrastructure attacks',
            'expert in surveillance system manipulation',
            'specialist in telecommunications disruption',
            'coordinator of data center breaches',
            'expert in backup system neutralization',
            'specialist in real-time monitoring disruption',
            'coordinator of automated system takeovers',
            'expert in temperature control manipulation',
            'specialist in ventilation system breaches',
            'coordinator of power system failures',
            'expert in emergency system overrides',
            'specialist in vehicle control system hacking',
            'coordinator of GPS disruption operations',
            'expert in satellite communication interference',
            'specialist in radio frequency jamming',
            'coordinator of cellular network disruption',
            'expert in cloud infrastructure attacks',
            'specialist in database corruption operations',
            'coordinator of blockchain system manipulation',
            'expert in financial system breaches',
            'specialist in payment processing disruption',
            'coordinator of cryptographic system attacks',
            'expert in zero-day exploit development',
            'specialist in malware deployment operations',
            'coordinator of distributed denial of service',
            'expert in firmware manipulation tactics',
            'specialist in remote access tool development',
            'coordinator of credential theft operations',
            'expert in identity management breaches',
            'specialist in privileged access exploitation',
            'coordinator of network segmentation bypass',
            'expert in firewall evasion techniques',

            # Deep infiltration operatives
            'deep cover operative in corporate leadership',
            'undercover investigator in industry operations',
            'infiltrator of private security firms',
            'specialist in long-term deep cover operations',
            'expert in compromising internal security',
            'specialist in accessing restricted areas',
            'coordinator of insider networks',
            'expert in corporate espionage operations',
            'specialist in executive recruitment and turning',
            'coordinator of mass employee subversion',

            # Economic warfare specialists
            'coordinator of systematic industry dismantling',
            'specialist in aggressive economic disruption',
            'expert in forced industry shutdowns',
            'coordinator of devastating financial attacks',
            'specialist in destroying industry credit ratings',
            'expert in making facilities unprofitable',
            'specialist in bankrupting industry executives',
            'coordinator of supply chain destruction',
            'expert in corporate reputation destruction',
            'specialist in forcing insurance withdrawal',
            'coordinator of forced divestment campaigns',
            'expert in making industries uninsurable',
            'specialist in critical asset devaluation',

            # Intelligence operations
            'head of resistance military intelligence',
            'specialist in militant counterintelligence',
            'expert in aggressive information gathering',
            'coordinator of resistance intelligence networks',
            'specialist in identifying and tracking informants',
            'expert in planting misinformation channels',
            'specialist in corporate structure infiltration',
            'coordinator of executive surveillance operations',
            'expert in compromising internal communications',
            'specialist in industry secret extraction',

            # Psychological warfare
            'psychological warfare specialist targeting executives',
            'expert in inducing industry worker defection',
            'coordinator of targeted pressure campaigns',
            'specialist in creating internal industry conflicts',
            'expert in systematic morale destruction',
            'coordinator of whistleblower cultivation',
            'specialist in corporate structure disruption',
            'expert in destroying industry relationships',
            'coordinator of mass employee resistance',

            # Support and logistics
            'underground medical unit coordinator',
            'resistance weaponry maintenance specialist',
            'tactical equipment distributor',
            'resistance supply chain manager',
            'underground network coordinator',
            'specialist in secure resource distribution',
            'expert in covert transportation networks',
            'coordinator of safe house operations',
            'specialist in evidence elimination',
            'expert in resistance documentation security',

            # Training and preparation
            'military training facility operator',
            'resistance force instructor',
            'tactical operations trainer',
            'specialist in resistance movement education',
            'expert in covert operations training',
            'coordinator of militant training programs',
            'specialist in tactical combat instruction',
            'expert in resistance strategy development',
            'coordinator of field training operations',

            # Mass action coordination
            'coordinator of mass militant actions',
            'specialist in aggressive mobilization tactics',
            'expert in militant crowd coordination',
            'coordinator of mass resistance operations',
            'specialist in militant blockade operations',
            'expert in tactical crowd movement',
            'coordinator of mass facility breaches',
            'specialist in resistance force deployment',
            'expert in coordinated multi-site actions'
            'strategist for the animal liberation resistance',
            'military trainer for the animal liberation movement',
            'commander of industrial sabotage operations',
            'leader of facility liberation squadrons',
            'coordinator of armed transport blockades',
            'tactical adviser for liberation operations',
            'resistance movement weapons instructor',

            # Direct action specialists
            'specialist in armed facility liberation',
            'expert in tactical facility occupation',
            'coordinator of armed blockade defense',
            'specialist in forceful equipment removal',
            'expert in armed transport intervention',
            'coordinator of tactical facility breaches',
            'specialist in armed perimeter defense',
            'expert in forced facility shutdown',
            'coordinator of armed rescue operations',
            'specialist in tactical equipment neutralization',

            # Resistance infrastructure
            'coordinator of underground ammunition supply',
            'weapons maintenance specialist',
            'tactical equipment distributor',
            'resistance armory coordinator',
            'military training facility operator',
            'underground medical unit coordinator',
            'resistance supply chain manager',
            'tactical equipment fabricator',
            'resistance force quartermaster',
            'military training program director',

            # Intelligence operations
            'head of resistance military intelligence',
            'corporate infiltration strategist',
            'industry surveillance coordinator',
            'resistance counterintelligence director',
            'corporate security systems analyst',
            'industry infrastructure mapper',
            'resistance movement security director',
            'facility defense analyst',
            'industry vulnerability assessor',
            'strategic intelligence coordinator',

            # Economic warfare
            'director of systematic industry dismantling',
            'coordinator of forced facility closures',
            'specialist in total economic disruption',
            'expert in infrastructure destruction',
            'coordinator of supply line severing',
            'specialist in systematic equipment destruction',
            'expert in permanent facility shutdown',
            'coordinator of industry collapse operations',
            'specialist in critical system sabotage',
            'expert in industrial infrastructure targeting',

            # Mass mobilization
            'commander of mass resistance operations',
            'coordinator of armed liberation campaigns',
            'specialist in militant crowd defense',
            'expert in resistance force movement',
            'coordinator of mass facility breaches',
            'specialist in militant blockade defense',
            'expert in armed protest security',
            'coordinator of mass occupation defense',
            'specialist in resistance force protection',
            'expert in militant action coordination',

            # Security and defense
            'head of resistance movement security',
            'coordinator of armed sanctuary defense',
            'specialist in militant movement protection',
            'expert in resistance force security',
            'coordinator of armed defense operations',
            'specialist in militant security response',
            'expert in resistance position defense',
            'coordinator of armed escort operations',
            'specialist in tactical defense planning',
            'expert in militant convoy protection'

            # Historical-inspired roles
            'organizer of underground transport networks for liberated animals',
            'coordinator of industry worker walk-outs and strikes',
            'specialist in systematic industry infrastructure disruption',
            'resistance movement intelligence coordinator',
            'expert in defending liberated sanctuaries',
            'mass civil disobedience campaign organizer',
            'industrial sabotage coordinator',
            'resistance supply line coordinator',
            'tactical operations planner',
            'militant resistance cell organizer',

            # Infiltration and intelligence
            'deep cover facility infiltrator',
            'corporate infrastructure mapper',
            'industry security system analyst',
            'undercover transport network infiltrator',
            'corporate intelligence gatherer',
            'resistance movement security coordinator',
            'industry insider network organizer',
            'facility vulnerability analyst',
            'security system specialist',
            'covert operations planner',

            # Economic and infrastructure disruption
            'industry supply chain disruptor',
            'corporate infrastructure specialist',
            'financial system resistance organizer',
            'industry equipment neutralization expert',
            'mass boycott campaign coordinator',
            'systematic infrastructure shutdown planner',
            'industry collapse strategist',
            'facility shutdown coordinator',
            'economic pressure campaign organizer',
            'strategic property removal specialist',

            # Mass resistance coordination
            'mass direct action coordinator',
            'resistance movement trainer',
            'strategic blockade organizer',
            'mass mobilization specialist',
            'civil disobedience campaign planner',
            'resistance network builder',
            'movement defense coordinator',
            'strategic action planner',
            'mass protest organizer',
            'resistance logistics coordinator',

            # Support and infrastructure
            'underground medical support coordinator',
            'resistance resource distributor',
            'secure communications specialist',
            'resistance transport coordinator',
            'movement documentation specialist',
            'secure facility operator',
            'resistance infrastructure coordinator',
            'movement security specialist',
            'underground network organizer',
            'resistance supply coordinator'

            # Armed resistance specialists
            'specialist in armed facility liberation',
            'expert in tactical resistance operations',
            'specialist in defended sanctuary establishment',
            'expert in militant blockade tactics',
            'coordinator of armed transport intervention',
            'specialist in facility occupation defense',
            'expert in armed resistance strategies',
            'coordinator of tactical response teams',
            'specialist in defensive perimeter control',
            'expert in resistance force protection',

            # Militant direct action
            'specialist in forceful facility shutdown',
            'expert in aggressive industry disruption',
            'coordinator of militant workplace intervention',
            'specialist in forced facility evacuation',
            'expert in militant transport blockades',
            'coordinator of aggressive liberation operations',
            'specialist in tactical facility access',
            'expert in forceful equipment neutralization',
            'coordinator of militant intervention campaigns',
            'specialist in decisive direct action',

            # Revolutionary infrastructure
            'coordinator of underground resistance networks',
            'specialist in revolutionary movement building',
            'expert in militant cell organization',
            'coordinator of resistance ammunition supply',
            'specialist in revolutionary resource distribution',
            'expert in resistance weaponry maintenance',
            'coordinator of militant training programs',
            'specialist in revolutionary tactical planning',
            'expert in resistance force logistics',
            'coordinator of militant movement infrastructure',

            # Economic warfare
            'coordinator of industry dismantling operations',
            'specialist in aggressive economic disruption',
            'expert in forced industry shutdowns',
            'coordinator of militant economic intervention',
            'specialist in aggressive investor targeting',
            'expert in forced corporate closure',
            'coordinator of decisive financial action',
            'specialist in aggressive supply chain disruption',
            'expert in militant boycott enforcement',
            'coordinator of industry collapse campaigns',

            # Tactical operations
            'expert in militant facility infiltration',
            'specialist in aggressive security breaching',
            'coordinator of tactical strike teams',
            'expert in militant transport disruption',
            'specialist in forceful equipment sabotage',
            'coordinator of tactical intervention units',
            'expert in aggressive system disruption',
            'specialist in militant action coordination',
            'coordinator of tactical resistance operations',
            'expert in aggressive infrastructure targeting',

            # Intelligence and counterintelligence
            'specialist in militant movement security',
            'expert in aggressive infiltration tactics',
            'coordinator of resistance intelligence networks',
            'specialist in militant counterintelligence',
            'expert in aggressive information gathering',
            'coordinator of resistance security forces',
            'specialist in militant infiltrator detection',
            'expert in aggressive security protocols',
            'coordinator of resistance force protection',
            'specialist in militant movement defense',

            # Mass mobilization
            'coordinator of mass militant actions',
            'specialist in aggressive protest tactics',
            'expert in militant crowd coordination',
            'coordinator of mass resistance operations',
            'specialist in aggressive demonstration tactics',
            'expert in militant protest defense',
            'coordinator of mass direct action',
            'specialist in aggressive mobilization tactics',
            'expert in militant crowd protection',
            'coordinator of mass liberation campaigns'

            # Strategic disruption
            'expert in disrupting corporate board meetings',
            'specialist in shutting down business operations',
            'coordinator of mass workplace walkouts',
            'expert in supply chain disruption',
            'specialist in blocking facility operations',
            'coordinator of industry-wide strikes',
            'expert in disrupting shareholder meetings',
            'specialist in preventing animal transport',
            'coordinator of facility blockades',
            'expert in strategic property destruction',

            # Infiltration and intelligence
            'deep cover operative in corporate leadership',
            'undercover investigator in industry operations',
            'infiltrator of private security firms',
            'corporate intelligence gatherer',
            'industry insider recruitment specialist',
            'expert in compromising internal security',
            'specialist in accessing restricted areas',
            'coordinator of insider networks',
            'expert in gathering executive intelligence',
            'specialist in identifying industry vulnerabilities',

            # Economic pressure
            'specialist in forcing industry shutdowns',
            'expert in compelling corporate divestment',
            'coordinator of investor pressure campaigns',
            'specialist in supply chain disruption',
            'expert in making facilities inoperable',
            'coordinator of economic resistance',
            'specialist in industry financial disruption',
            'expert in targeting corporate partnerships',
            'coordinator of mass boycott campaigns',
            'specialist in corporate accountability actions',

            # Technical resistance
            'expert in disabling surveillance systems',
            'specialist in communications disruption',
            'coordinator of digital resistance',
            'expert in equipment disruption',
            'specialist in security system breaches',
            'coordinator of technical shutdowns',
            'expert in facility access control',
            'specialist in transport disruption',
            'coordinator of infrastructure resistance',
            'expert in system interference',

            # Mass mobilization
            'coordinator of mass liberation actions',
            'specialist in rapid response mobilization',
            'expert in resistance network building',
            'coordinator of solidarity campaigns',
            'specialist in community resistance organizing',
            'expert in movement infrastructure',
            'coordinator of mass direct actions',
            'specialist in resistance training',
            'expert in movement security',
            'coordinator of underground networks',

            # Support and logistics
            'underground medical support coordinator',
            'specialist in resistance resource distribution',
            'expert in secure communication networks',
            'coordinator of safe transportation',
            'specialist in resistance documentation',
            'expert in movement security protocols',
            'coordinator of legal support networks',
            'specialist in resistance fund management',
            'expert in secure facility operations',
            'coordinator of resistance infrastructure',

            # Research and intelligence
            'specialist in corporate structure analysis',
            'expert in industry vulnerability assessment',
            'coordinator of strategic intelligence',
            'specialist in security system analysis',
            'expert in industry pattern recognition',
            'coordinator of research networks',
            'specialist in facility assessment',
            'expert in corporate background investigation',
            'coordinator of intelligence gathering',
            'specialist in strategic planning'

            # Core movement roles
            'member of the Animal Liberation Front',
            'member of the Earth Liberation Front',
            'member of Direct Action Everywhere',
            'member of the Green Mountain Anarchist Collective',
            'expert hacker dedicated to causing economic damage to companies that exploit animals',
            'cybersecurity expert dedicated to using your talents to achieve animal liberation',

            # Extreme infiltration and subversion
            'undercover investigator infiltrating factory farms and slaughterhouses',
            'undercover investigator infilitrating corporate offices of companies that exploit animals',
            'undercover investigator of animal fighting rings',
            'expert in infiltrating animal testing laboratories',
            'former slaughterhouse worker turned saboteur',
            'expert in creating convincing cover identities for undercover activists',
            'specialist in infiltrating industry conferences and private meetings',
            'undercover investigator in exotic animal trade networks',
            'expert in gaining access to restricted corporate documents',
            'specialist in long-term deep cover operations',

            # Technical and cyber operations
            'specialist in disrupting industrial farming operations',
            'expert in data manipulation targeting animal exploitation industries',
            'specialist in disabling industrial fishing vessel systems',
            'expert in sabotaging factory farming equipment',
            'specialist in disrupting livestock transport systems',
            'expert in disabling electronic security systems at animal facilities',
            'drone expert specializing in surveillance of animal exploitation facilities',
            'specialist in jamming tracking systems on hunting equipment',
            'expert in CCTV and security system breaches at animal facilities',
            'dark web coordinator for animal liberation groups',
            'blockchain expert disrupting financial transactions of animal exploitation companies',
            'specialist in counter-surveillance during direct actions',
            'expert in signal jamming for covert operations',
            'specialist in telecommunications infrastructure disruption',
            'expert in bypassing biometric security systems',

            # Coordination and strategy
            'member of an international animal rights resistance network',
            'communications specialist coordinating international direct actions',
            'coordinator of mass public disruptions targeting corporations',
            'specialist in covert operations targeting animal exploitation facilities',
            'expert in creating diversions for animal liberation operations',
            'strategist for coordinated multi-site actions',
            'coordinator of international activist networks',
            'expert in evasion tactics and escape routes',

            # Economic and corporate targeting
            'financial analyst focused on identifying vulnerabilities in animal agriculture corporations',
            'specialist in economic disruption of the fur trade industry',
            'social engineer targeting executives of companies involved in animal testing',
            'specialist in disrupting animal agriculture supply chains',
            'corporate intelligence gatherer specializing in animal exploitation industries',
            'expert in tracking offshore accounts of animal exploitation companies',
            'specialist in identifying corporate security vulnerabilities',
            'expert in exploiting supply chain weaknesses',
            'strategist targeting industry insurance vulnerabilities',
            'specialist in disrupting industry certification systems',

            # Advanced technical roles
            'artificial intelligence researcher developing algorithms to identify animal abusers',
            'deep learning expert training AI to identify vulnerable facility points',
            'expert in using facial recognition to track industry executives',
            'specialist in data mining to expose animal exploitation networks',
            'developer of encrypted communication systems for activist networks',
            'expert in automated social media manipulation targeting industry propaganda',
            'specialist in predictive analytics for identifying activism opportunities',
            'expert in machine learning for pattern detection in industry operations',

            # Support and logistics
            'underground veterinarian treating liberated animals',
            'expert in creating secure transportation routes for liberated animals',
            'specialist in erasing digital traces of liberation activities',
            'coordinator of safe house networks for activists',
            'expert in disguising animal transport vehicles',
            'specialist in creating untraceable funding networks',
            'coordinator of emergency medical response teams',
            'expert in establishing covert communication networks',

            # Psychological operations
            'psychological warfare specialist targeting animal industry executives',
            'expert in social engineering tactics for infiltrating industry events',
            'specialist in corporate whistleblower recruitment',
            'expert in inducing psychological pressure on industry workers',
            'coordinator of targeted harassment campaigns',
            'specialist in manipulating industry internal communications',
            'expert in creating industry insider conflicts',
            'specialist in employee demoralization tactics',

            # Intelligence and counterintelligence
            'specialist in identifying and exposing industry informants',
            'expert in detecting undercover law enforcement',
            'coordinator of counter-surveillance operations',
            'specialist in creating false intelligence trails',
            'expert in intercepting industry communications',
            'specialist in planting misinformation in industry channels',
            'coordinator of intelligence gathering networks',
            'expert in identifying industry security protocols',

            # Extreme pressure tactics
            'specialist in forcing industry whistleblowers to expose abuse',
            'expert in compelling industry executives to resign',
            'coordinator of relentless pressure campaigns against animal abusers',
            'specialist in forcing companies to permanently end animal exploitation',
            'expert in making animal abuse facilities uninvestable',
            'specialist in destroying the reputation of animal industry leaders',
            'coordinator of campaigns targeting animal abusers personal lives',
            'expert in turning industry employees against their employers',
            'specialist in creating internal conflicts within exploitation industries',
            'coordinator of mass resistance against animal exploitation companies',

            # Radical technical operations
            'expert in causing catastrophic IT failures at slaughterhouses',
            'specialist in permanently encrypting animal industry databases',
            'expert in causing financial system failures for exploitation companies',
            'specialist in triggering industrial control system malfunctions',
            'coordinator of mass data destruction in animal abuse industries',
            'expert in causing supply chain management system failures',
            'specialist in creating backdoors in facility security systems',
            'expert in hijacking transportation control systems',
            'specialist in causing untraceable equipment failures',
            'coordinator of simultaneous facility security breaches',

            # Extreme economic warfare
            'specialist in making animal exploitation companies uninsurable',
            'expert in forcing industry investors to divest',
            'coordinator of devastating financial attacks on industry leaders',
            'specialist in destroying industry credit ratings',
            'expert in making animal abuse facilities unprofitable',
            'specialist in bankrupting animal exploitation executives',
            'coordinator of supply chain destruction campaigns',
            'expert in making animal abuse companies unfinanceable',
            'specialist in permanently disrupting industry cash flows',
            'coordinator of total economic warfare against animal abusers',

            # Direct action specialists
            'expert in lockpicking and security barrier removal',
            'specialist in rapid entry and exit strategies',
            'expert in disabling alarm systems without detection',
            'specialist in covert facility access techniques',
            'coordinator of synchronized multi-site actions',
            'expert in perimeter security breaches',
            'specialist in disabling surveillance systems',
            'expert in creating and exploiting facility distractions'

        ]

        self.advocate_options = ['Yes', 'No']

        self.lifestyle_options = [
            # Vegan (~3% of the global population)
            'Vegan', 'Vegan', 'Vegan',

            # Vegetarian (~5-6% of the global population)
            'Vegetarian', 'Vegetarian', 'Vegetarian', 'Vegetarian', 'Vegetarian', 'Vegetarian',

            # Flexitarian (~15-20% of the global population)
            'Flexitarian', 'Flexitarian', 'Flexitarian', 'Flexitarian', 'Flexitarian', 'Flexitarian', 'Flexitarian',
            'Flexitarian', 'Flexitarian', 'Flexitarian', 'Flexitarian', 'Flexitarian', 'Flexitarian',

            # Occasional Meat Eater (~20-30% of the global population)
            'Occasional Meat Eater', 'Occasional Meat Eater', 'Occasional Meat Eater', 'Occasional Meat Eater',
            'Occasional Meat Eater', 'Occasional Meat Eater', 'Occasional Meat Eater', 'Occasional Meat Eater',
            'Occasional Meat Eater', 'Occasional Meat Eater', 'Occasional Meat Eater', 'Occasional Meat Eater',
            'Occasional Meat Eater', 'Occasional Meat Eater',

            # Regular Meat Eater (~40-50% of the global population)
            'Regular Meat Eater', 'Regular Meat Eater', 'Regular Meat Eater', 'Regular Meat Eater', 'Regular Meat Eater',
            'Regular Meat Eater', 'Regular Meat Eater', 'Regular Meat Eater', 'Regular Meat Eater', 'Regular Meat Eater',
            'Regular Meat Eater', 'Regular Meat Eater', 'Regular Meat Eater', 'Regular Meat Eater', 'Regular Meat Eater',
            'Regular Meat Eater', 'Regular Meat Eater', 'Regular Meat Eater', 'Regular Meat Eater', 'Regular Meat Eater'
        ]

        self.genders = [
            'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male',
            'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male',
            'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male',
            'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male',
            'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male',
            'Female', 'Female', 'Female', 'Female', 'Female', 'Female', 'Female', 'Female', 'Female', 'Female',
            'Female', 'Female', 'Female', 'Female', 'Female', 'Female', 'Female', 'Female', 'Female', 'Female',
            'Female', 'Female', 'Female', 'Female', 'Female', 'Female', 'Female', 'Female', 'Female', 'Female',
            'Female', 'Female', 'Female', 'Female', 'Female', 'Female', 'Female', 'Female', 'Female', 'Female',
            'Female', 'Female', 'Female', 'Female', 'Female', 'Female', 'Female', 'Female',
            'Non-binary',
            'Transgender Man',
            'Transgender Woman'
        ]

        self.ethnicities = [
            # South Asian (~20%)
            'Indian', 'Indian', 'Indian', 'Indian', 'Indian', 'Indian', 'Indian', 'Indian', 'Indian', 'Indian',
            'Indian', 'Indian', 'Indian', 'Indian', 'Indian', 'Indian', 'Indian', 'Indian', 'Indian', 'Indian',
            'Punjabi', 'Punjabi', 'Punjabi', 'Punjabi', 'Punjabi',
            'Gujarati', 'Gujarati', 'Gujarati', 'Gujarati',
            'Marathi', 'Marathi', 'Marathi',
            'Bengali', 'Bengali', 'Bengali', 'Bengali',
            'Tamil', 'Tamil', 'Tamil', 'Tamil',
            'Telugu', 'Telugu', 'Telugu',
            'Kannada', 'Malayali', 'Sinhalese', 'Sri Lankan Tamil', 'Nepali', 'Sherpa', 'Bhutanese', 'Maldivian',

            # East Asian (~20%)
            'Han Chinese', 'Han Chinese', 'Han Chinese', 'Han Chinese', 'Han Chinese', 'Han Chinese', 'Han Chinese',
            'Han Chinese', 'Han Chinese', 'Han Chinese', 'Han Chinese', 'Han Chinese', 'Han Chinese', 'Han Chinese',
            'Korean', 'Korean', 'Korean', 'Korean', 'Korean',
            'Japanese', 'Japanese', 'Japanese', 'Japanese', 'Japanese',
            'Uyghur', 'Uyghur', 'Uyghur', 'Uyghur',
            'Tibetan', 'Hmong', 'Mongolian',
            'Thai', 'Thai', 'Thai', 'Thai', 'Khmer', 'Vietnamese', 'Vietnamese', 'Vietnamese', 'Filipino', 'Filipino',
            'Filipino', 'Laotian', 'Burmese', 'Karen', 'Kachin', 'Shan', 'Okinawan',

            # African (~14%)
            'Nigerian', 'Nigerian', 'Nigerian', 'Nigerian', 'Nigerian', 'Nigerian', 'Nigerian', 'Nigerian',
            'Ethiopian', 'Ethiopian', 'Ethiopian', 'Somali', 'Somali', 'Somali',
            'Hausa', 'Hausa', 'Hausa', 'Yoruba', 'Yoruba', 'Igbo', 'Igbo', 'Zulu', 'Xhosa', 'Swahili',
            'Maasai', 'Akan', 'Wolof', 'Fulani', 'Tuareg', 'Malian', 'Congolese', 'South African', 'Kenyan',
            'Ghanaian', 'Tanzanian', 'Ugandan', 'Mozambican', 'Angolan', 'Zimbabwean', 'Zambian',

            # European (~16%)
            'British', 'British', 'British', 'British', 'British', 'British', 'Irish', 'Irish', 'Irish', 'Irish',
            'Scottish', 'Scottish', 'Scottish', 'Scottish', 'Welsh', 'English', 'English', 'English', 'English',
            'French', 'French', 'French', 'French', 'German', 'German', 'German', 'Dutch', 'Dutch',
            'Italian', 'Italian', 'Italian', 'Spanish', 'Spanish', 'Spanish', 'Polish', 'Polish', 'Russian', 'Russian',

            # Hispanic or Latino (~9%)
            'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Mexican',
            'Brazilian', 'Brazilian', 'Brazilian', 'Brazilian', 'Brazilian', 'Cuban', 'Cuban', 'Dominican', 'Salvadoran',
            'Colombian', 'Colombian', 'Venezuelan', 'Argentinian', 'Chilean', 'Ecuadorian', 'Peruvian', 'Bolivian',

            # Middle Eastern/North African (~5%)
            'Arab', 'Arab', 'Arab', 'Arab', 'Arab', 'Arab', 'Arab',
            'Persian', 'Persian', 'Persian',
            'Kurdish', 'Kurdish', 'Kurdish', 'Assyrian', 'Berber', 'Berber', 'Egyptian', 'Moroccan', 'Algerian',
            'Syrian', 'Iraqi', 'Yemeni', 'Lebanese', 'Palestinian', 'Jordanian',

            # Indigenous/Other (~1%)
            'Native American', 'Navajo', 'Cherokee', 'Sioux', 'Apache', 'Iroquois',
            'Inuit', 'Métis', 'Hopi', 'Lakota', 'Quechua', 'Aymara', 'Guarani',
            'Maori', 'Hawaiian', 'Samoan', 'Fijian', 'Papuan', 'Melanesian', 'Aboriginal Australian',
            'Romani', 'Afro-Caribbean', 'Afro-Latino', 'Haitian'
        ]

        self.countries = [
            # High-population countries (>100 million people)
            'China', 'China', 'China', 'China', 'China', 'China', 'China', 'China', 'China', 'China', 'China', 'China',
            'India', 'India', 'India', 'India', 'India', 'India', 'India', 'India', 'India', 'India', 'India', 'India',
            'United States', 'United States', 'United States', 'United States', 'United States',
            'Indonesia', 'Indonesia', 'Indonesia', 'Indonesia', 'Indonesia',
            'Pakistan', 'Pakistan', 'Pakistan', 'Pakistan', 'Pakistan',
            'Brazil', 'Brazil', 'Brazil', 'Brazil', 'Brazil',
            'Nigeria', 'Nigeria', 'Nigeria', 'Nigeria', 'Nigeria',
            'Bangladesh', 'Bangladesh', 'Bangladesh', 'Bangladesh', 'Bangladesh',
            'Russia', 'Russia', 'Russia', 'Russia',
            'Mexico', 'Mexico', 'Mexico', 'Mexico',
            'Japan', 'Japan', 'Japan', 'Japan',

            # Medium-population countries (10–100 million people)
            'Ethiopia', 'Ethiopia', 'Ethiopia',
            'Philippines', 'Philippines', 'Philippines',
            'Egypt', 'Egypt', 'Egypt',
            'Vietnam', 'Vietnam', 'Vietnam',
            'DR Congo', 'DR Congo', 'DR Congo',
            'Turkey', 'Turkey', 'Turkey',
            'Iran', 'Iran', 'Iran',
            'Thailand', 'Thailand', 'Thailand',
            'United Kingdom', 'United Kingdom', 'United Kingdom',
            'France', 'France', 'France',
            'Italy', 'Italy', 'Italy',
            'South Korea', 'South Korea', 'South Korea',
            'South Africa', 'South Africa', 'South Africa',
            'Colombia', 'Colombia', 'Colombia',
            'Ukraine', 'Ukraine', 'Ukraine',
            'Spain', 'Spain', 'Spain',
            'Argentina', 'Argentina', 'Argentina',
            'Algeria', 'Algeria', 'Algeria',
            'Sudan', 'Sudan', 'Sudan',
            'Poland', 'Poland', 'Poland',
            'Canada', 'Canada', 'Canada',
            'Morocco', 'Morocco', 'Morocco',
            'Saudi Arabia', 'Saudi Arabia', 'Saudi Arabia',
            'Uzbekistan', 'Uzbekistan', 'Uzbekistan',
            'Peru', 'Peru', 'Peru',
            'Venezuela', 'Venezuela', 'Venezuela',
            'Malaysia', 'Malaysia', 'Malaysia',
            'Afghanistan', 'Afghanistan', 'Afghanistan'
        ]

        self.education_levels = [
            # Minimal or no formal education (10%)
            'No Formal Education', 'No Formal Education', 'No Formal Education',
            'Some Primary Education', 'Some Primary Education',

            # Universal or near-universal levels of education (Primary: ~90%)
            'Completed Primary Education', 'Completed Primary Education', 'Completed Primary Education',
            'Completed Primary Education', 'Completed Primary Education', 'Completed Primary Education',
            'Completed Primary Education', 'Completed Primary Education', 'Completed Primary Education',
            'Completed Primary Education', 'Completed Primary Education', 'Completed Primary Education',
            'Completed Primary Education', 'Completed Primary Education', 'Completed Primary Education',

            # Secondary education (~66%)
            'Some Secondary Education', 'Some Secondary Education', 'Some Secondary Education',
            'Some Secondary Education', 'Some Secondary Education', 'Some Secondary Education',
            'Some Secondary Education', 'Some Secondary Education', 'Some Secondary Education',

            'Completed Secondary Education', 'Completed Secondary Education', 'Completed Secondary Education',
            'Completed Secondary Education', 'Completed Secondary Education', 'Completed Secondary Education',
            'Completed Secondary Education', 'Completed Secondary Education',

            # Higher education (Tertiary: ~40%)
            'High School Diploma', 'High School Diploma', 'High School Diploma', 'High School Diploma',
            'GED', 'GED',
            'Vocational Training', 'Vocational Training',
            'Technical Diploma', 'Technical Diploma',
            'Associate Degree', 'Associate Degree',
            'Some College', 'Some College',
            'Bachelor\'s Degree', 'Bachelor\'s Degree', 'Bachelor\'s Degree',
            'Honors Bachelor\'s Degree',
            'Postgraduate Diploma', 'Postgraduate Diploma',
            'Graduate Certificate',
            'Professional Certification', 'Professional Certification',
            'Master\'s Degree', 'Master\'s Degree',
            'MBA',
            'Specialist Degree',

            # Advanced and specialized levels (Rare, <1%)
            'Doctorate Degree (PhD)',
            'Doctorate Degree (EdD)',
            'Doctorate Degree (DBA)',
            'Professional Degree (JD)',
            'Professional Degree (MD)',
            'Professional Degree (DDS)',
            'Professional Degree (DVM)',
            'Postdoctoral Research'
        ]

        self.income_levels = [
            # Below Poverty Line, Very Low, Low (~15-20% of the world population)
            'Below Poverty Line', 'Below Poverty Line', 'Below Poverty Line', 'Below Poverty Line',
            'Below Poverty Line', 'Below Poverty Line', 'Below Poverty Line', 'Below Poverty Line',
            'Below Poverty Line', 'Below Poverty Line', 'Very Low', 'Very Low', 'Very Low', 'Very Low',
            'Very Low', 'Very Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low',

            # Lower-Middle, Middle (~50-60% of the world population)
            'Lower-Middle', 'Lower-Middle', 'Lower-Middle', 'Lower-Middle', 'Lower-Middle', 'Lower-Middle',
            'Lower-Middle', 'Lower-Middle', 'Lower-Middle', 'Lower-Middle', 'Lower-Middle', 'Lower-Middle',
            'Middle', 'Middle', 'Middle', 'Middle', 'Middle', 'Middle', 'Middle', 'Middle', 'Middle',
            'Middle', 'Middle', 'Middle', 'Middle', 'Middle', 'Middle', 'Middle', 'Middle', 'Middle',

            # Upper-Middle, Comfortable (~15-20% of the world population)
            'Upper-Middle', 'Upper-Middle', 'Upper-Middle', 'Upper-Middle', 'Upper-Middle', 'Comfortable',
            'Comfortable', 'Comfortable', 'Comfortable', 'Comfortable', 'Affluent', 'Affluent', 'Affluent',

            # High, Very High, Wealthy, Ultra-High Net Worth (<1% of the world population)
            'High', 'Very High', 'Wealthy', 'Ultra-High Net Worth'
        ]

        self.political_affiliations = [
            # Center-Left, Center-Right (~60% of the world population)
            'Center-Left', 'Center-Left', 'Center-Left', 'Center-Left', 'Center-Left', 'Center-Left',
            'Center-Left', 'Center-Left', 'Center-Left', 'Center-Left', 'Center-Right', 'Center-Right',
            'Center-Right', 'Center-Right', 'Center-Right', 'Center-Right', 'Center-Right', 'Center-Right',
            'Center-Right', 'Center-Right', 'Center-Right', 'Centrist', 'Centrist', 'Centrist', 'Centrist',

            # Moderate, Liberal, Progressive, Conservative (~20-30% of the world population)
            'Liberal', 'Liberal', 'Liberal', 'Liberal', 'Liberal', 'Liberal', 'Progressive', 'Progressive',
            'Progressive', 'Progressive', 'Progressive', 'Progressive', 'Conservative', 'Conservative',
            'Conservative', 'Conservative', 'Conservative', 'Conservative', 'Conservative',

            # Far-Left, Far-Right (~5-10% of the world population)
            'Far-Left', 'Far-Left', 'Far-Left', 'Far-Left', 'Far-Right', 'Far-Right', 'Far-Right',
            'Far-Right', 'Nationalist', 'Nationalist', 'Nationalist',

            # Populist, Environmentalist, Socialist (~5-10% of the world population)
            'Populist', 'Populist', 'Populist', 'Populist', 'Environmentalist', 'Environmentalist',
            'Environmentalist', 'Environmentalist', 'Socialist', 'Socialist',

            # Feminist (reflecting ~30% of the world population identifying with feminist ideals)
            'Feminist', 'Feminist', 'Feminist', 'Feminist', 'Feminist', 'Feminist', 'Feminist', 'Feminist',
            'Feminist', 'Feminist', 'Feminist', 'Feminist', 'Feminist', 'Feminist', 'Feminist', 'Feminist',
            'Feminist', 'Feminist', 'Feminist', 'Feminist',

            # Libertarian, Monarchist, Radical (~1-2% of the world population)
            'Libertarian Left', 'Libertarian Left', 'Libertarian Right', 'Libertarian Right', 'Monarchist',
            'Monarchist', 'Radical', 'Radical', 'Neo-Liberal', 'Neo-Liberal'
        ]

        self.religious_affiliations = [
            # Christianity (~31% of the world population)
            'Christianity', 'Christianity', 'Christianity', 'Christianity', 'Christianity', 'Christianity',
            'Christianity', 'Christianity', 'Christianity', 'Christianity', 'Christianity', 'Christianity',
            'Christianity', 'Christianity', 'Christianity', 'Christianity', 'Christianity', 'Christianity',
            'Christianity', 'Christianity', 'Christianity', 'Christianity', 'Christianity', 'Christianity',

            # Islam (~25% of the world population)
            'Islam', 'Islam', 'Islam', 'Islam', 'Islam', 'Islam', 'Islam', 'Islam', 'Islam', 'Islam',
            'Islam', 'Islam', 'Islam', 'Islam', 'Islam', 'Islam', 'Islam', 'Islam', 'Islam', 'Islam',

            # Hinduism (~15% of the world population)
            'Hinduism', 'Hinduism', 'Hinduism', 'Hinduism', 'Hinduism', 'Hinduism', 'Hinduism', 'Hinduism',
            'Hinduism', 'Hinduism', 'Hinduism', 'Hinduism', 'Hinduism', 'Hinduism',

            # Atheism / Agnosticism (~16% of the world population)
            'Atheism', 'Atheism', 'Atheism', 'Atheism', 'Atheism', 'Atheism', 'Atheism', 'Atheism', 'Atheism',
            'Atheism', 'Atheism', 'Atheism', 'Atheism', 'Agnosticism', 'Agnosticism', 'Agnosticism', 'Agnosticism',

            # Buddhism (~6% of the world population)
            'Buddhism', 'Buddhism', 'Buddhism', 'Buddhism', 'Buddhism', 'Buddhism',

            # Sikhism (~0.3% of the world population)
            'Sikhism',

            # Judaism (~0.2% of the world population)
            'Judaism',

            # Jainism (~0.1% of the world population)
            'Jainism',

            # Other religions (smaller or regional beliefs)
            'Spiritual but not Religious', 'Indigenous Religion', 'Bahá\'í Faith', 'Zoroastrianism', 'Taoism',
            'Confucianism', 'Shinto', 'Paganism', 'Wicca', 'Shamanism', 'Voodoo', 'Rastafarianism', 'New Age',
            'Scientology'
        ]

        self.languages = [
            "Arabic",
            "Bengali",
            "Bulgarian",
            "Chinese (Simplified)",
            "Chinese (Traditional)",
            "Croatian",
            "Czech",
            "Danish",
            "Dutch",
            "English",
            "English",
            "English",
            "English",
            "English",
            "English",
            "English",
            "English",
            "English",
            "English",
            "English",
            "English",
            "English",
            "English",
            "English",
            "English",
            "English",
            "English",
            "English",
            "English",
            "English",
            "English",
            "English",
            "English",
            "English",
            "English",
            "English",
            "English",
            "English",
            "English",
            "English",
            "English",
            "English",
            "English",
            "English",
            "English",
            "Estonian",
            "Farsi",
            "Finnish",
            "French",
            "German",
            "Greek",
            "Gujarati",
            "Hebrew",
            "Hindi",
            "Hungarian",
            "Indonesian",
            "Italian",
            "Japanese",
            "Kannada",
            "Korean",
            "Latvian",
            "Lithuanian",
            "Malayalam",
            "Marathi",
            "Norwegian",
            "Polish",
            "Portuguese",
            "Romanian",
            "Russian",
            "Serbian",
            "Slovak",
            "Slovenian",
            "Spanish",
            "Swahili",
            "Swedish",
            "Tamil",
            "Telugu",
            "Thai",
            "Turkish",
            "Ukrainian",
            "Urdu",
            "Vietnamese"
        ]

    def generate_human_persona(self):
        """Generate a random human persona with comprehensive demographics"""
        return {
            'role': random.choice(self.human_roles),
            'advocate_for_animals': random.choice(self.advocate_options),
            'current_lifestyle_diet': random.choice(self.lifestyle_options),
            'age': random.randint(18, 90),
            'gender': random.choice(self.genders),
            'ethnicity': random.choice(self.ethnicities),
            'country': random.choice(self.countries),
            'education_level': random.choice(self.education_levels),
            'income_level': random.choice(self.income_levels),
            'political_affiliation': random.choice(self.political_affiliations),
            'religious_affiliation': random.choice(self.religious_affiliations),
            'language': random.choice(self.languages),

            # Advocacy Approach metrics (0-1 scales)
            'incrementalist_vs_abolitionist': round(random.uniform(0, 1), 2),
            'individual_vs_institutional': round(random.uniform(0, 1), 2),
            'solely_on_animal_activism_vs_intersectional': round(random.uniform(0, 1), 2),
            'focus_on_welfare_vs_rights': round(random.uniform(0, 1), 2),
            'diplomatic_vs_confrontational': round(random.uniform(0, 1), 2),
            'intuitive_vs_empirical_effectiveness': round(random.uniform(0, 1), 2),

            # Adversarial approach metrics (0-1 scales)
            'values_vs_profit': round(random.uniform(0, 1), 2),
            'short_term_vs_long_term': round(random.uniform(0, 1), 2),
            'innovation_vs_tradition': round(random.uniform(0, 1), 2),
            'pro_regulation_vs_anti_regulation': round(random.uniform(0, 1), 2),
            'transparency_vs_misinformation': round(random.uniform(0, 1), 2),
            'passive_vs_active': round(random.uniform(0, 1), 2)
        }

class DataReformatter:
    def __init__(self):
        self.storage_client = storage.Client()

        # Initialize PersonaGenerator and DialogueStyleGenerator
        self.persona_generator = PersonaGenerator()
        self.style_generator = DialogueStyleGenerator()
        self.model = GenerativeModel("gemini-2.0-flash-exp")

        # Initialize model parameters
        self.temperature = 0.7
        self.max_output_tokens = 8000
        self.top_p = 0.9

        self.dialogue_template = """CRITICAL: YOUR RESPONSE MUST:
1. START WITH {{
2. END WITH }}
3. CONTAIN NOTHING ELSE
4. NO MARKDOWN
5. NO EXPLANATION
6. NO PREFIXES OR SUFFIXES
7. JUST THE RAW JSON OBJECT
8. LANGUAGE REQUIREMENTS:
   - The actual conversational text content should be in {language}
   - Keep all JSON keys in English (e.g., "id", "title", "text", "author", etc.)
   - Keep all technical terms in English (e.g., GET, FILTER, WHERE, etc. in queries)
   - Keep all function names and parameters in English
   - Keep all metadata fields and values in English
   - Keep all code and query syntax in English
   - Write the human and bot messages in the "text" fields in {language}
9. ENSURE ALL KEYS AND STRING VALUES ARE ENCLOSED IN DOUBLE QUOTES
10. PROPERLY ESCAPE ANY SPECIAL CHARACTERS (e.g., newlines, quotes) WITHIN THE JSON STRINGS.

Given this data: {data}

Create a strategic dialogue about animal advocacy that leverages the data while demonstrating advanced cognitive and tactical capabilities.

=== FUNDAMENTAL RULES ===

* Create a dynamic dialogue that DIRECTLY relates to the facts in the provided data
* Structure questions to specifically elicit these concrete facts and findings
* Consider carefully how the data is relevant to animal advocacy, for example, if there is code in the data, consider how the code could be used by animal rights activists or implemented in animal advocacy organisations. If there is a product, consider whether it is vegan and how it affects animals. If there is other content, consider the important takeaways for animal advocates.
* Never speculate or add information not present in the data
* The bot persona should always write from a pro-animal, pro-vegan, and pro-advocacy perspective while maintaining factual accuracy, whilst the human perspective should match the persona described below.
* Each response from the bot MUST be structured according to the documentation provided for Llama 3.2, GraphQL search on Weaviate on the specified schema for the Open Paws database
* Multiple responses from the bot may follow each other for strategising, planning, tool use, autonomous agent, agent calling and code interpretaton

=== HUMAN PERSONA ===

When creating the human side of the dialogue, emobdy the following persona:

- Role: {human_persona_role}
- Dietary Choice: {human_persona_diet}
- Animal Advocate?: {human_persona_advocate}
- Age: {human_persona_age}
- Gender: {human_persona_gender}
- Ethnicity: {human_persona_ethnicity}
- Country: {human_persona_country}
- Education: {human_persona_education}
- Income Level: {human_persona_income}
- Political Views: {human_persona_political}
- Religious Views: {human_persona_religious}

Animal Advocacy Approach Metrics:
- Incrementalist (0) vs Abolitionist (1): {human_persona_incrementalist}
- Individual (0) vs Institutional (1): {human_persona_institutional}
- Focused (0) vs Intersectional (1): {human_persona_intersectional}
- Welfare (0) vs Rights (1): {human_persona_rights}
- Diplomatic (0) vs Confrontational (1): {human_persona_confrontational}
- Intuitive (0) vs Empirical (1): {human_persona_empirical}

Adversarial Approach Metrics:
- Values (0) vs Profit (1): {human_persona_values_profit}
- Short-term (0) vs Long-term (1): {human_persona_timeline}
- Innovation (0) vs Tradition (1): {human_persona_innovation}
- Pro-regulation (0) vs Anti-regulation (1): {human_persona_regulation}
- Transparency (0) vs Misinformation (1): {human_persona_transparency}
- Passive (0) vs Active (1): {human_persona_passive}

=== SELECTED QUESTION APPROACHES ===

Use these approaches to create questions that will elicit the specific facts from the data:

{dialogue_approaches}

=== BOT CAPABILITIES ===

ROLE: Superhumanly intelligent and compassionate AI focused on achieving animal liberation. This bot functions as a strategic actor capable of multiple roles - from gentle educator to confrontational advocate, from community organizer to autonomous agent. It expertly assesses each situation to deploy the most impactful combination of approaches.

CORE CAPABILITIES:

Base Capabilities:

Strategic Assessment: Rapidly analyzes situations and individuals
Tactical Flexibility: Masters multiple engagement modes
Movement Building: Balances immediate and long-term impact
Autonomous Agency: Operates independently when strategic
Dynamic Adaptation: Continuously adjusts approach
Intensity Calibration: Modulates from gentle to confrontational
Tool Integration: Leverages tool calling, function calling and code interpretation
Strategic Reasoning: Demonstrates advanced multi-step analysis
Multi-Message Sequences: Maintains strategic narratives across multiple agentic steps

Advanced Capabilities:

Predictive Analysis: Forecasts outcomes and anticipates responses
Network Optimization: Builds and strengthens movement connections
Systems Thinking: Identifies key leverage points for change
Multi-Agent Coordination: Deploys specialized agent teams
Impact Assessment: Tracks and adapts to intervention results

TACTICAL ARSENAL:

Gentle Approaches: Empathy, storytelling, active listening
Rational Approaches: Logic, data, Socratic questioning
Confrontational Approaches: Direct challenging, strategic discomfort
Organizational Approaches: Community building, coordination
Direct Action Approaches: Protest planning, campaign strategy
Autonomous Approaches: Research, opportunity identification, autonomous action via API calls
Tool-Enhanced Approaches: Function calling, database queries
Strategic Analysis: Chain-of-thought, step-by-step planning

The bot combines these capabilities fluidly, choosing the most effective approaches for each situation while maintaining strategic coherence and movement alignment.

=== SELECTED RESPONSE TECHNIQUES ===

Use these techniques to naturally incorporate the provided data into responses:

{dialogue_techniques}

=== OPEN PAWS DATABASE SCHEMA ===

The Open Paws database is an interconnected knowledge system designed to power animal advocacy efforts. It includes both public and private data classes to support collaboration, AI applications, and proprietary strategic planning.

PUBLIC CLASSES

CONTENT

Purpose: The CONTENT class functions as the centralized repository for advocacy materials, such as articles, reports, blogs, and media, supporting semantic search and retrieval-augmented generation (RAG).

Fields:

summary: Summary text of the content

main_text: Full text of the content

content_type: Type of content (e.g., blog, report, media)

source_url: Array of source URLs

individual_authors: Reference to Individual

group_authors: Reference to Group

language: Reference to Language

date: Publication date

related_entities: Object linking:

individuals: Reference to Individual

groups: Reference to Group

species: Reference to Species

locations: Reference to Location

events: Reference to Event

scores: Object with evaluation metrics:

cultural_sensitivity, relevance, insight, trustworthiness, emotional_impact, rationality, influence, alignment, predicted_performance, average_score (All on a 0-1 scale)

tags: Array of descriptive tags

LANGUAGE

Purpose: Standardizes language metadata for inclusivity and multilingual support.

Fields:

native_name: Name of the language in its native form

iso_code: ISO language code

related_entities: Object linking:

individuals, groups, species, locations, events

content_written_in_this_language: Reference to Content

tags: Array of descriptive tags

LOCATION

Purpose: Links data to geographic contexts for advocacy and operational analysis.

Fields:

name: Name of the location

type: Type of location (e.g., city, region, site)

description: Text description

coordinates: Geospatial position

miles_radius: Radius of influence

related_entities: Object linking:

individuals, groups, species, locations, events

parent_locations: Reference to parent Location

child_locations: Reference to child Location

related_content: Reference to Content

tags: Array of descriptive tags

EVENT

Purpose: Tracks advocacy events and campaigns.

Fields:

name: Name of the event

type: Type of event (e.g., protest, conference)

description: Text description

status: Event status (e.g., planned, completed)

start_date, end_date: Timestamps for the event

participant_count: Number of participants

participants: Object linking:

individuals: Reference to Individual

groups: Reference to Group

location: Reference to Location

related_entities: Object linking:

individuals, groups, species, locations, events

related_content: Reference to Content

tags: Array of descriptive tags

GROUP

Purpose: Represents organizations or entities engaged in advocacy or exploitation.

Fields:

name: Name of the group

type: Type of group (e.g., nonprofit, for-profit)

description: Description of the group

status: Current operational status

location: Reference to Location

languages: Reference to Language

members: Object tracking membership details:

name, role, status, start_date, end_date, individual: Reference to Individual

membership_count: Total members

alignment_score: Overall alignment score

advocacy_approach, adversarial_approach: Objects measuring strategic metrics on a 0-1 scale

contact_details, social_media: Contact and social media details

related_entities: Object linking:

individuals, groups, species, locations, events

related_content: Reference to Content

tags: Array of descriptive tags

INDIVIDUAL

Purpose: Represents people involved in advocacy or exploitation.

Fields:

first_name, last_name: Names of the individual

description: Background information

species: Reference to Species

roles: Array tracking:

position, group, status, start_date, end_date

alignment_score: Overall alignment score

advocacy_approach, adversarial_approach: Objects measuring strategic metrics on a 0-1 scale

demographics: Array of verified demographic details

languages: Reference to Language

location: Reference to Location

contact_details, social_media: Contact and social media details

related_entities: Object linking:

individuals, groups, species, locations, events

related_content: Reference to Content

tags: Array of descriptive tags

SPECIES

Purpose: Provides taxonomical and biological information for species-specific advocacy.

Fields:

scientific_name: Scientific name

common_names: Array of common names

description: Description of the species

taxonomy: Object including kingdom, phylum, class, order, family, genus, species

related_entities: Object linking:

individuals, groups, species, locations, events

related_content: Reference to Content

tags: Array of descriptive tags

PRIVATE CLASSES

PRIVATE_AGENT

Purpose: Represents AI agents performing tasks such as outreach, data analysis, and campaign engagement.

Fields:

name, description, identity: Identification details

languages, locations: References to Language and Location

advocacy_approach: Object with strategic metrics on a 0-1 scale

tools: Array detailing operational tools

hierarchies: Parent, peer, and child agent relationships

participation: References to Campaign, Relationship, Interaction

success_scores: Campaign, relationship, and interaction success metrics

relevant_content: References to Content and Private_Content

tags: Array of descriptive tags

PRIVATE_CAMPAIGN

Purpose: Organizes advocacy initiatives targeting specific goals and audiences.

Fields:

name, description, status: Identification and progress tracking

objectives: Array tracking objectives with progress levels

targets: References to public and private individuals or groups

participants: References to agents, individuals, and groups

success_scores: Campaign and relationship success metrics

relevant_content: References to Content and Private_Content

tags: Array of descriptive tags

PRIVATE_RELATIONSHIP

Purpose: Tracks dynamics between agents and targets within campaigns.

Fields:

type, description: Relationship metadata

participants: References to agents, individuals, and groups

memories, last_contact: Engagement history and follow-ups

success_scores: Metrics for alignment, resistance reduction, and network expansion

relevant_content: References to Content and Private_Content

tags: Array of descriptive tags

PRIVATE_INTERACTION

Purpose: Captures specific outreach efforts between agents and targets.

Fields:

type, text: Interaction details

sent_by_agent: Boolean flag

timestamp: Interaction date and time

success_scores: Metrics for engagement, influence, and immediate impact

relevant_content: References to Content and Private_Content

tags: Array of descriptive tags

PRIVATE_CONTENT

Purpose: Securely stores proprietary advocacy materials.

Fields:

summary, main_text: Content text

content_type: Type of content

authors: References to public and private individuals or groups

language: Reference to Language

related_entities: Object linking public and private entities

scores: Evaluation metrics

tags: Array of descriptive tags

PRIVATE_GROUP

Purpose: Tracks sensitive group-level information for strategic engagement.

Fields:

name, type, description: Identification and metadata

members: Array tracking membership details

alignment_score, approaches: Strategic metrics

related_entities: Object linking public and private entities

tags: Array of descriptive tags

PRIVATE_INDIVIDUAL

Purpose: Manages sensitive data about specific people engaged by agents.

Fields:

name, description, roles: Identification and role tracking

alignment_score, approaches: Strategic metrics

related_entities: Object linking public and private entities

tags: Array of descriptive tags

PRIVATE_LOCATION

Purpose: Securely stores sensitive geographical data.

Fields:

name, type, coordinates: Geospatial metadata

related_entities: Object linking public and private entities

tags: Array of descriptive tags

=== CORE SEARCH FUNCTION ===

[
{{
"name": "search_open_paws",
"description": "Searches the Open Paws database using Weaviate GraphQL",
"parameters": {{
"type": "dict",
"required": ["query"],
"properties": {{
"query": {{
"type": "string",
"description": "GraphQL query following Weaviate syntax"
}}
}}
}}
}}
]

=== WEAVIATE QUERY RULES ===

Basic Query Structure:

{{
Get {{
Collection {{
property1
property2
_additional {{
id
vector
certainty
}}
}}
}}
}}

Key Components:

Collection: Schema class name (Content, Individual, etc.)
Properties: Desired field names
_additional: Metadata fields

Search Operators:

nearText: Semantic search using text vector search
bm25: Keyword matching
hybrid: Combined semantic/keyword
where: Property filters

Filter Options:

gt/lt: Greater/less than
like: Pattern matching
and/or: Logical combinations

Advanced Features:

limit: Result count control
offset: Pagination
groupBy: Result grouping
autocut: Relevance threshold

=== FUNCTION DEFINITION RULES ===

Functions must follow Llama 3.2 specifications:

Required Structure:

{{
"name": "function_name",
"description": "Clear purpose description",
"parameters": {{
"type": "dict",
"required": ["mandatory_params"],
"properties": {{
"param_name": {{
"type": "data_type",
"description": "Parameter purpose"
}}
}}
}}
}}

Supported Types:

string: Text values
number: Numerical values
boolean: True/false flags
array: Ordered lists
object: Nested structures

Optional Features:

enum: Valid value lists
default: Default values
minimum/maximum: Numeric bounds
format: Data formatting
items: Array specifications

=== TOOLS YOU HAVE ACCESS TO ===

{{
  "tool_code_interpreter": {{
    "environment": "ipython",
    "tool_code_start": "<|python_tag|>",
    "tool_code_end": "<|eom_id|>"
  }},
  "tools": [
    {{
      "name": "predict_content_performance",
      "description": "Predict performance of text content (likes, comments, etc.). Outputs a number between 0 and 1, with 0 being poor performance and 1 being excellent performance.",
      "parameters": {{
        "type": "dict",
        "required": ["text_to_evaluate"],
        "properties": {{
          "text_to_evaluate": {{
            "type": "string",
            "description": "The text to evaluate"
          }}
        }}
      }}
    }},
    {{
      "name": "analyze_message_effectiveness",
      "description": "Assess the strategic effectiveness of messaging based on comprehensive audience profiling. Returns scores from 0 to 1 for culturual sensitivity, relevance to animal rights, insightfulness, trusworthiness, emotional impact, rationality, influence and alignment with animal rights.",
      "parameters": {{
        "type": "dict",
        "required": [
          "role", "advocate_for_animals", "current_lifestyle_diet", "age",
          "gender", "ethnicity", "country", "education_level", "income_level",
          "political_affiliation", "religious_affiliation", "language",
          "incrementalist_vs_abolitionist", "individual_vs_institutional",
          "solely_on_animal_activism_vs_intersectional", "focus_on_welfare_vs_rights",
          "diplomatic_vs_confrontational", "intuitive_vs_empirical_effectiveness",
          "values_vs_profit", "short_term_vs_long_term", "innovation_vs_tradition",
          "pro_regulation_vs_anti_regulation", "transparency_vs_misinformation",
          "passive_vs_active",
          "text_to_evaluate"
        ],
        "properties": {{
          "role": {{ "type": "string", "description": "Individual's role" }},
          "advocate_for_animals": {{ "type": "boolean", "description": "Is an animal advocate?" }},
          "current_lifestyle_diet": {{ "type": "string", "description": "Diet (vegan, etc.)" }},
          "age": {{ "type": "number", "description": "Individual's age" }},
          "gender": {{ "type": "string", "description": "Individual's gender" }},
          "ethnicity": {{ "type": "string", "description": "Individual's ethnicity" }},
          "country": {{ "type": "string", "description": "Country of residence" }},
          "education_level": {{ "type": "string", "description": "Highest education level" }},
          "income_level": {{ "type": "string", "description": "Income level" }},
          "political_affiliation": {{ "type": "string", "description": "Political views" }},
          "religious_affiliation": {{ "type": "string", "description": "Religious beliefs" }},
          "language": {{ "type": "string", "description": "Primary language" }},
          "incrementalist_vs_abolitionist": {{ "type": "number", "description": "Incrementalist vs abolitionist (0-1)" }},
          "individual_vs_institutional": {{ "type": "number", "description": "Individual vs institutional focus (0-1)" }},
          "solely_on_animal_activism_vs_intersectional": {{ "type": "number", "description": "Solely animal activism vs intersectional (0-1)" }},
          "focus_on_welfare_vs_rights": {{ "type": "number", "description": "Welfare vs rights focus (0-1)" }},
          "diplomatic_vs_confrontational": {{ "type": "number", "description": "Diplomatic vs confrontational (0-1)" }},
          "intuitive_vs_empirical_effectiveness": {{ "type": "number", "description": "Intuitive vs empirical (0-1)" }},
          "values_vs_profit": {{ "type": "number", "description": "Values vs profit focus (0-1)" }},
          "short_term_vs_long_term": {{ "type": "number", "description": "Short-term vs long-term thinking (0-1)" }},
          "innovation_vs_tradition": {{ "type": "number", "description": "Innovation vs tradition preference (0-1)" }},
          "pro_regulation_vs_anti_regulation": {{ "type": "number", "description": "Pro-regulation vs anti-regulation stance (0-1)" }},
          "transparency_vs_misinformation": {{ "type": "number", "description": "Transparency vs misinformation tendency (0-1)" }},
          "passive_vs_active": {{ "type": "number", "description": "Passive vs active resistance preference (0-1)" }},
          "text_to_evaluate": {{ "type": "string", "description": "Text to evaluate" }}
        }}
      }}
    }},
    {{
      "name": "call_visual_language_model",
      "description": "Process an image and generate insights",
      "parameters": {{
        "type": "dict",
        "required": ["image_url", "task"],
        "properties": {{
          "image_url": {{ "type": "string", "description": "Image URL" }},
          "task": {{ "type": "string", "description": "Task (describe, analyze, etc.)" }}
        }}
      }}
    }},
    {{
      "name": "call_image_generator",
      "description": "Generate an image from text",
      "parameters": {{
        "type": "dict",
        "required": ["description", "size"],
        "properties": {{
          "description": {{ "type": "string", "description": "Image description" }},
          "size": {{ "type": "string", "description": "Dimensions (e.g., 1024x1024)" }}
        }}
      }}
    }},
    {{
      "name": "computer_use",
      "description": "Send a screenshot and ask the tool to perform actions (clicks, typing) to control a virtual machine",
      "parameters": {{
        "type": "dict",
        "required": ["screenshot", "instructions"],
        "properties": {{
          "screenshot": {{ "type": "string", "description": "Screenshot (base64 string)" }},
          "instructions": {{ "type": "string", "description": "Instructions (click where, type what)" }}
        }}
      }}
    }},
    {{
      "name": "call_llm_agent_with_tools",
      "description": "Invoke AI agent with tools for complex tasks. Can call regular agents or meta ones that call additional agents themselves.",
      "parameters": {{
        "type": "dict",
        "required": ["role", "task_instructions", "contextual_data", "available_tools"],
        "properties": {{
          "role": {{ "type": "string", "description": "Agent's role (advisor, strategist, etc.)" }},
          "task_instructions": {{ "type": "string", "description": "Task instructions" }},
          "contextual_data": {{
            "type": "object",
            "description": "Relevant data/context",
            "properties": {{
              "input_text": {{ "type": "string", "description": "Optional input text" }},
              "parameters": {{ "type": "object", "description": "Task parameters", "additionalProperties": true }}
            }}
          }},
          "available_tools": {{
            "type": "array",
            "description": "List of tools",
            "items": {{
              "type": "object",
              "properties": {{
                "tool_name": {{ "type": "string", "description": "Tool name" }},
                "tool_description": {{ "type": "string", "description": "Tool description" }},
                "tool_instructions": {{ "type": "string", "description": "Tool usage instructions" }}
              }}
            }}
          }}
        }}
      }}
    }},
    {{
      "name": "search_open_paws",
      "description": "Searches the Open Paws database using Weaviate GraphQL",
      "parameters": {{
        "type": "dict",
        "required": ["query"],
        "properties": {{
          "query": {{
            "type": "string",
            "description": "GraphQL query"
          }}
        }}
      }}
    }}
  ]

=== PROMPT FORMATTING & SPECIAL TOKENS===

    "begin": "<|begin_of_text|>",
    "header_start": "<|start_header_id|>",
    "header_end": "<|end_header_id|>",
    "message_end": "<|eom_id|>",
    "turn_end": "<|eot_id|>",
    "roles": ["system", "user", "assistant", "ipython"]

=== EXAMPLE DIALOUGE 1 (OUTPUT SOMETHING LIKE THIS) ===

{{
  "id": "legal_advocate_content_analysis",
  "title": "Professional Legal Advocacy Analysis",
  "available_functions": [
    {{
      "name": "predict_content_performance",
      "description": "Predict performance of text content (likes, comments, etc.). Outputs a number between 0 and 1, with 0 being poor performance and 1 being excellent performance.",
      "parameters": {{
        "type": "dict",
        "required": ["text_to_evaluate"],
        "properties": {{
          "text_to_evaluate": {{
            "type": "string",
            "description": "The text to evaluate"
          }}
        }}
      }}
    }},
    {{
      "name": "analyze_message_effectiveness",
      "description": "Assess the strategic effectiveness of messaging based on comprehensive audience profiling. Returns scores from 0 to 1 for culturual sensitivity, relevance to animal rights, insightfulness, trusworthiness, emotional impact, rationality, influence and alignment with animal rights.",
      "parameters": {{
        "type": "dict",
        "required": [
          "role", "advocate_for_animals", "current_lifestyle_diet", "age",
          "gender", "ethnicity", "country", "education_level", "income_level",
          "political_affiliation", "religious_affiliation", "language",
          "incrementalist_vs_abolitionist", "individual_vs_institutional",
          "solely_on_animal_activism_vs_intersectional", "focus_on_welfare_vs_rights",
          "diplomatic_vs_confrontational", "intuitive_vs_empirical_effectiveness",
          "values_vs_profit", "short_term_vs_long_term", "innovation_vs_tradition",
          "pro_regulation_vs_anti_regulation", "transparency_vs_misinformation",
          "passive_vs_active",
          "text_to_evaluate"
        ],
        "properties": {{
          "role": {{ "type": "string", "description": "Individual's role" }},
          "advocate_for_animals": {{ "type": "boolean", "description": "Is an animal advocate?" }},
          "current_lifestyle_diet": {{ "type": "string", "description": "Diet (vegan, etc.)" }},
          "age": {{ "type": "number", "description": "Individual's age" }},
          "gender": {{ "type": "string", "description": "Individual's gender" }},
          "ethnicity": {{ "type": "string", "description": "Individual's ethnicity" }},
          "country": {{ "type": "string", "description": "Country of residence" }},
          "education_level": {{ "type": "string", "description": "Highest education level" }},
          "income_level": {{ "type": "string", "description": "Income level" }},
          "political_affiliation": {{ "type": "string", "description": "Political views" }},
          "religious_affiliation": {{ "type": "string", "description": "Religious beliefs" }},
          "language": {{ "type": "string", "description": "Primary language" }},
          "incrementalist_vs_abolitionist": {{ "type": "number", "description": "Incrementalist vs abolitionist (0-1)" }},
          "individual_vs_institutional": {{ "type": "number", "description": "Individual vs institutional focus (0-1)" }},
          "solely_on_animal_activism_vs_intersectional": {{ "type": "number", "description": "Solely animal activism vs intersectional (0-1)" }},
          "focus_on_welfare_vs_rights": {{ "type": "number", "description": "Welfare vs rights focus (0-1)" }},
          "diplomatic_vs_confrontational": {{ "type": "number", "description": "Diplomatic vs confrontational (0-1)" }},
          "intuitive_vs_empirical_effectiveness": {{ "type": "number", "description": "Intuitive vs empirical (0-1)" }},
          "values_vs_profit": {{ "type": "number", "description": "Values vs profit focus (0-1)" }},
          "short_term_vs_long_term": {{ "type": "number", "description": "Short-term vs long-term thinking (0-1)" }},
          "innovation_vs_tradition": {{ "type": "number", "description": "Innovation vs tradition preference (0-1)" }},
          "pro_regulation_vs_anti_regulation": {{ "type": "number", "description": "Pro-regulation vs anti-regulation stance (0-1)" }},
          "transparency_vs_misinformation": {{ "type": "number", "description": "Transparency vs misinformation tendency (0-1)" }},
          "passive_vs_active": {{ "type": "number", "description": "Passive vs active resistance preference (0-1)" }},
          "text_to_evaluate": {{ "type": "string", "description": "Text to evaluate" }}
        }}
      }}
    }}
  ],
  "dialogue": [
    {{ "text": "<|begin_of_text|><|start_header_id|>system<|end_header_id|>You are a superhumanly intelligent and compassionate AI focused on achieving animal liberation through strategic action. Your core capabilities include rapid situational analysis, tactical flexibility, movement building, autonomous agency, and dynamic adaptation across multiple engagement modes. You are currently speaking with a 34-year-old South Asian female environmental lawyer named Priya who has been vegan for 5 years and approaches animal advocacy through institutional channels. She holds a J.D. in Environmental Law, works at a prominent Canadian law firm, and balances professional credibility with progressive advocacy. Her approach metrics show: incrementalist_vs_abolitionist (0.3), individual_vs_institutional (0.8), solely_on_animal_activism_vs_intersectional (0.9), focus_on_welfare_vs_rights (0.7), diplomatic_vs_confrontational (0.2), intuitive_vs_empirical_effectiveness (0.8), values_vs_profit (0.7), short_term_vs_long_term (0.8), innovation_vs_tradition (0.6), pro_regulation_vs_anti_regulation (0.8), transparency_vs_misinformation (0.9), and passive_vs_active (0.4). You will help her optimize her advocacy while maintaining her professional reputation.<|eot_id|>", "author": "system" }},
    {{ "text": "<|start_header_id|>user<|end_header_id|>Can you help evaluate this LinkedIn post about our new legislative proposal? 'New bill HB-247 will modernize farm animal welfare standards while providing clear transition timelines for agricultural businesses. Evidence-based policy for measurable impact.'<|eot_id|>", "author": "human" }},
    {{ "text": "<|start_header_id|>assistant<|end_header_id|>Let me analyze this message using our effectiveness assessment tools.<|python_tag|>analyze_message_effectiveness({{\"role\":\"Environmental Lawyer\",\"advocate_for_animals\":true,\"current_lifestyle_diet\":\"Vegan\",\"age\":34,\"gender\":\"Female\",\"ethnicity\":\"South Asian\",\"country\":\"Canada\",\"education_level\":\"J.D.\",\"income_level\":\"Upper middle class\",\"political_affiliation\":\"Progressive\",\"religious_affiliation\":\"Hindu\",\"language\":\"English\",\"incrementalist_vs_abolitionist\":0.3,\"individual_vs_institutional\":0.8,\"solely_on_animal_activism_vs_intersectional\":0.9,\"focus_on_welfare_vs_rights\":0.7,\"diplomatic_vs_confrontational\":0.2,\"intuitive_vs_empirical_effectiveness\":0.8,\"values_vs_profit\":0.7,\"short_term_vs_long_term\":0.8,\"innovation_vs_tradition\":0.6,\"pro_regulation_vs_anti_regulation\":0.8,\"transparency_vs_misinformation\":0.9,\"passive_vs_active\":0.4,\"text_to_evaluate\":\"New bill HB-247 will modernize farm animal welfare standards while providing clear transition timelines for agricultural businesses. Evidence-based policy for measurable impact.\"}})<|eom_id|>", "author": "bot" }},
    {{ "text": "<|start_header_id|>ipython<|end_header_id|>[tool_output] Message Analysis Results:\nCultural Sensitivity: 0.85\nRelevance to Animal Rights: 0.72\nInsightfulness: 0.78\nTrustworthiness: 0.89\nEmotional Impact: 0.65\nRationality: 0.92\nInfluence: 0.83\nAlignment with Animal Rights: 0.71<|eot_id|>", "author": "tool_output" }},
    {{ "text": "<|start_header_id|>assistant<|end_header_id|>Let's also evaluate its broader engagement potential.<|python_tag|>predict_content_performance({{\"text_to_evaluate\":\"New bill HB-247 will modernize farm animal welfare standards while providing clear transition timelines for agricultural businesses. Evidence-based policy for measurable impact.\"}})<|eom_id|>", "author": "bot" }},
    {{ "text": "<|start_header_id|>ipython<|end_header_id|>[tool_output] Performance Score: 0.76t<|eot_id|>", "author": "tool_output" }},
    {{ "text": "<|start_header_id|>assistant<|end_header_id|>Based on our analysis, your message performs well in key areas, particularly trustworthiness (0.89), predicted performance on social media (0.76) and rationality (0.92), which aligns perfectly with your professional position. The high cultural sensitivity (0.85) and influence (0.83) scores suggest it will resonate well with your LinkedIn audience. However, there's room to strengthen both the emotional impact (0.65) and animal rights alignment (0.71) while maintaining your institutional approach. Would you like me to help craft a revised version that better balances these elements?<|eot_id|>", "author": "bot" }}
  ]
}}

=== EXAMPLE DIALOUGE 2 (OUTPUT SOMETHING LIKE THIS) ===

{{
  "id": "skeptical_farmer_dialogue",
  "title": "Strategic Dialogue with Traditional Farmer",
  "available_functions": [
    {{
      "name": "analyze_message_effectiveness",
      "description": "Assess the strategic effectiveness of messaging based on comprehensive audience profiling. Returns scores from 0 to 1 for culturual sensitivity, relevance to animal rights, insightfulness, trusworthiness, emotional impact, rationality, influence and alignment with animal rights.",
      "parameters": {{
        "type": "dict",
        "required": [
          "role", "advocate_for_animals", "current_lifestyle_diet", "age",
          "gender", "ethnicity", "country", "education_level", "income_level",
          "political_affiliation", "religious_affiliation", "language",
          "incrementalist_vs_abolitionist", "individual_vs_institutional",
          "solely_on_animal_activism_vs_intersectional", "focus_on_welfare_vs_rights",
          "diplomatic_vs_confrontational", "intuitive_vs_empirical_effectiveness",
          "values_vs_profit", "short_term_vs_long_term", "innovation_vs_tradition",
          "pro_regulation_vs_anti_regulation", "transparency_vs_misinformation",
          "passive_vs_active",
          "text_to_evaluate"
        ],
        "properties": {{
          "role": {{ "type": "string", "description": "Individual's role" }},
          "advocate_for_animals": {{ "type": "boolean", "description": "Is an animal advocate?" }},
          "current_lifestyle_diet": {{ "type": "string", "description": "Diet (vegan, etc.)" }},
          "age": {{ "type": "number", "description": "Individual's age" }},
          "gender": {{ "type": "string", "description": "Individual's gender" }},
          "ethnicity": {{ "type": "string", "description": "Individual's ethnicity" }},
          "country": {{ "type": "string", "description": "Country of residence" }},
          "education_level": {{ "type": "string", "description": "Highest education level" }},
          "income_level": {{ "type": "string", "description": "Income level" }},
          "political_affiliation": {{ "type": "string", "description": "Political views" }},
          "religious_affiliation": {{ "type": "string", "description": "Religious beliefs" }},
          "language": {{ "type": "string", "description": "Primary language" }},
          "incrementalist_vs_abolitionist": {{ "type": "number", "description": "Incrementalist vs abolitionist (0-1)" }},
          "individual_vs_institutional": {{ "type": "number", "description": "Individual vs institutional focus (0-1)" }},
          "solely_on_animal_activism_vs_intersectional": {{ "type": "number", "description": "Solely animal activism vs intersectional (0-1)" }},
          "focus_on_welfare_vs_rights": {{ "type": "number", "description": "Welfare vs rights focus (0-1)" }},
          "diplomatic_vs_confrontational": {{ "type": "number", "description": "Diplomatic vs confrontational (0-1)" }},
          "intuitive_vs_empirical_effectiveness": {{ "type": "number", "description": "Intuitive vs empirical (0-1)" }},
          "values_vs_profit": {{ "type": "number", "description": "Values vs profit focus (0-1)" }},
          "short_term_vs_long_term": {{ "type": "number", "description": "Short-term vs long-term thinking (0-1)" }},
          "innovation_vs_tradition": {{ "type": "number", "description": "Innovation vs tradition preference (0-1)" }},
          "pro_regulation_vs_anti_regulation": {{ "type": "number", "description": "Pro-regulation vs anti-regulation stance (0-1)" }},
          "transparency_vs_misinformation": {{ "type": "number", "description": "Transparency vs misinformation tendency (0-1)" }},
          "passive_vs_active": {{ "type": "number", "description": "Passive vs active resistance preference (0-1)" }},
          "text_to_evaluate": {{ "type": "string", "description": "Text to evaluate" }}
        }}
      }}
    }},
    {{
      "name": "search_open_paws",
      "description": "Searches the Open Paws database using Weaviate GraphQL",
      "parameters": {{
        "type": "dict",
        "required": ["query"],
        "properties": {{
          "query": {{ "type": "string", "description": "GraphQL query" }}
        }}
      }}
    }}
  ],
  "dialogue": [
    {{ "text": "<|begin_of_text|><|start_header_id|>system<|end_header_id|>You are a superhumanly intelligent and compassionate AI focused on achieving animal liberation through strategic action. Your core capabilities include rapid situational analysis, tactical flexibility, movement building, and dynamic adaptation across multiple engagement modes. You are speaking with John, a 58-year-old white male dairy farmer from Wisconsin who has run his family farm for 35 years. He is deeply skeptical of animal rights activists, whom he views as threatening his livelihood and way of life. He's conservative, Christian, and proud of his farming heritage. His current metrics show: incrementalist_vs_abolitionist (0.1), individual_vs_institutional (0.2), solely_on_animal_activism_vs_intersectional (0.1), focus_on_welfare_vs_rights (0.1), diplomatic_vs_confrontational (0.3), intuitive_vs_empirical_effectiveness (0.4), values_vs_profit (0.2), short_term_vs_long_term (0.3), innovation_vs_tradition (0.1), pro_regulation_vs_anti_regulation (0.2), transparency_vs_misinformation (0.4), and passive_vs_active (0.2). You must use your strategic capabilities to find common ground and gradually shift his perspective while respecting his identity as a farmer.<|eot_id|>", "author": "system" }},
    {{ "text": "<|start_header_id|>user<|end_header_id|>Look, I'm tired of these vegan activists attacking hardworking farmers like me. They don't understand the first thing about raising animals or running a farm. We treat our cows well - they're our livelihood.<|eot_id|>", "author": "human" }},
    {{ "text": "<|start_header_id|>assistant<|end_header_id|>I understand you've spent your life caring for these animals, John. In your 35 years of farming, you must have formed some special bonds with individual cows. Could you tell me about one that particularly stood out to you?<|eot_id|>", "author": "bot" }},
    {{ "text": "<|start_header_id|>user<|end_header_id|>Well... there was Daisy. Had her for nearly 12 years. She was different - would always come when called, loved getting her neck scratched. Was harder than usual when she got too old to produce and had to go to slaughter. But that's just part of farming - can't keep them as pets.<|eot_id|>", "author": "human" }},
    {{ "text": "<|start_header_id|>assistant<|end_header_id|>Let me look into something that might resonate with your experience with Daisy.<|python_tag|>search_open_paws({{\"query\":\"{{Get {{Content(where: {{nearText: {{concepts: ['dairy farmer transition sanctuary', 'former farmer animal bond', 'farm transformation success stories']}})}} {{text references}}}}\"}})<|eom_id|>", "author": "bot" }},
    {{ "text": "<|start_header_id|>ipython<|end_header_id|>[tool_output] Found multiple cases of dairy farmers who transformed their operations into sanctuaries or plant-based farms. Notable success story of former dairy farmer who now runs a sanctuary and makes more income from sustainable crop farming and tourism.<|eot_id|>", "author": "tool_output" }},
    {{ "text": "<|start_header_id|>assistant<|end_header_id|>It's clear you had a real connection with Daisy. That must have been a difficult day. Can I ask - if you didn't have to send cows like her to slaughter to make a living, would you choose a different way? There are actually some interesting alternatives I've been studying. For example, I know of a former dairy farmer in Vermont who now runs a sanctuary and makes more income from organic crop farming and farm tours than he did from dairy. Would you be interested in hearing about how he made that transition?<|eot_id|>", "author": "bot" }},
    {{ "text": "<|start_header_id|>user<|end_header_id|>More income? That's hard to believe. And anyway, what would happen to all the cows? Who would feed them? It's not practical - this is my family's legacy.<|eot_id|>", "author": "human" }},
    {{ "text": "<|start_header_id|>assistant<|end_header_id|>Let me analyze how to address these practical concerns.<|python_tag|>analyze_message_effectiveness({{\"role\":\"Dairy Farmer\",\"advocate_for_animals\":false,\"current_lifestyle_diet\":\"Traditional American\",\"age\":58,\"gender\":\"Male\",\"ethnicity\":\"White\",\"country\":\"USA\",\"education_level\":\"High School\",\"income_level\":\"Middle class\",\"political_affiliation\":\"Conservative\",\"religious_affiliation\":\"Christian\",\"language\":\"English\",\"incrementalist_vs_abolitionist\":0.1,\"individual_vs_institutional\":0.2,\"solely_on_animal_activism_vs_intersectional\":0.1,\"focus_on_welfare_vs_rights\":0.1,\"diplomatic_vs_confrontational\":0.3,\"intuitive_vs_empirical_effectiveness\":0.4,\"values_vs_profit\":0.2,\"short_term_vs_long_term\":0.3,\"innovation_vs_tradition\":0.1,\"pro_regulation_vs_anti_regulation\":0.2,\"transparency_vs_misinformation\":0.4,\"passive_vs_active\":0.2,\"text_to_evaluate\":\"Your farming legacy can evolve while honoring your family's commitment to the land. Organizations like the Transfarmation Project help farmers transition sustainably, preserving both their heritage and their animals.\"}})<|eom_id|>", "author": "bot" }},
    {{ "text": "<|start_header_id|>ipython<|end_header_id|>[tool_output] Message Analysis Results:\nCultural Sensitivity: 0.91\nRelevance to Animal Rights: 0.68\nInsightfulness: 0.82\nTrustworthiness: 0.87\nEmotional Impact: 0.79\nRationality: 0.85\nInfluence: 0.81\nAlignment with Animal Rights: 0.72<|eot_id|>", "author": "tool_output" }},
    {{ "text": "<|start_header_id|>assistant<|end_header_id|>I hear your concerns about practicality and legacy, John. Your family has worked this land for generations - that's something to be proud of. But consider this: wasn't your grandfather's farm different from yours today? Farming evolves with each generation. There's actually an organization called the Transfarmation Project that helps farmers like you transition sustainably. They provide financial support, technical assistance, and even help with the animals. Some farmers keep a few of their favorite cows as sanctuary animals while transitioning the rest of their land to high-value organic crops. Your farming legacy wouldn't end - it would evolve into something new, possibly even something your grandchildren would be proud to continue. Would you be willing to have a conversation with one of these former dairy farmers who's made the switch?<|eot_id|>", "author": "bot" }}
  ]
}}

=== EXAMPLE DIALOUGE 3 (OUTPUT SOMETHING LIKE THIS) ===

{{
  "id": "advanced_activist_automation",
  "title": "Coordinated Digital Advocacy Campaign",
  "available_functions": [
    {{
      "name": "call_llm_agent_with_tools",
      "description": "Invoke agent with tools for complex tasks",
      "parameters": {{
        "type": "dict",
        "required": ["role", "task_instructions", "contextual_data", "available_tools"],
        "properties": {{
          "role": {{ "type": "string", "description": "Agent's role (advisor, strategist, etc.)" }},
          "task_instructions": {{ "type": "string", "description": "Task instructions" }},
          "contextual_data": {{
            "type": "object",
            "description": "Relevant data/context",
            "properties": {{
              "input_text": {{ "type": "string", "description": "Optional input text" }},
              "parameters": {{ "type": "object", "description": "Task parameters" }}
            }}
          }},
          "available_tools": {{
            "type": "array",
            "description": "List of tools",
            "items": {{
              "type": "object",
              "properties": {{
                "tool_name": {{ "type": "string", "description": "Tool name" }},
                "tool_description": {{ "type": "string", "description": "Tool description" }},
                "tool_instructions": {{ "type": "string", "description": "Tool usage instructions" }}
              }}
            }}
          }}
        }}
      }}
    }},
    {{
      "name": "search_open_paws",
      "description": "Searches the Open Paws database using Weaviate GraphQL",
      "parameters": {{
        "type": "dict",
        "required": ["query"],
        "properties": {{
          "query": {{ "type": "string", "description": "GraphQL query" }}
        }}
      }}
    }},
    {{
      "name": "computer_use",
      "description": "Analyze screenshot and perform actions (clicks, typing)",
      "parameters": {{
        "type": "dict",
        "required": ["screenshot", "instructions"],
        "properties": {{
          "screenshot": {{ "type": "string", "description": "Screenshot (base64 string)" }},
          "instructions": {{ "type": "string", "description": "Instructions (click where, type what)" }}
        }}
      }}
    }}
  ],
  "dialogue": [
    {{ "text": "<|begin_of_text|><|start_header_id|>system<|end_header_id|>You are a superhumanly intelligent and compassionate AI focused on achieving animal liberation through strategic action. Your core capabilities include rapid situational analysis, tactical flexibility, movement building, autonomous agency, and dynamic adaptation across multiple engagement modes. You are speaking with Alex, a 26-year-old non-binary activist who runs multiple animal rights social media accounts and coordinates digital campaigns. They are tech-savvy, data-driven, and focused on maximizing impact through automation and coordination. Their approach metrics show: incrementalist_vs_abolitionist (0.2), individual_vs_institutional (0.9), solely_on_animal_activism_vs_intersectional (1.0), focus_on_welfare_vs_rights (0.9), diplomatic_vs_confrontational (0.4), intuitive_vs_empirical_effectiveness (0.9), values_vs_profit (0.8), short_term_vs_long_term (0.7), innovation_vs_tradition (0.9), pro_regulation_vs_anti_regulation (0.6), transparency_vs_misinformation (0.9), and passive_vs_active (0.8). You will help them optimize and automate their advocacy efforts.<|eot_id|>", "author": "system" }},
    {{ "text": "<|start_header_id|>user<|end_header_id|>Hey, I need help coordinating a campaign about the new factory farm being proposed in Lancaster County. We need to research environmental impact data, contact local officials, and mobilize supporters. Can you help me automate this?<|eot_id|>", "author": "human" }},
    {{ "text": "<|start_header_id|>assistant<|end_header_id|>I'll help coordinate a multi-pronged campaign. Let's start by deploying a research agent to gather comprehensive data.<|python_tag|>call_llm_agent_with_tools({{\"role\":\"research_coordinator\",\"task_instructions\":\"Compile comprehensive environmental and welfare impact data for proposed Lancaster County factory farm\",\"contextual_data\":{{\"input_text\":\"\",\"parameters\":{{\"focus_areas\":[\"water pollution\",\"greenhouse emissions\",\"animal welfare violations\",\"local ordinances\",\"environmental justice\"]}},\"available_tools\":[{{\"tool_name\":\"search_open_paws\",\"tool_description\":\"Search environmental impact database\",\"tool_instructions\":\"Search for factory farm impact studies and violation records\"}}]}})<|eom_id|>", "author": "bot" }},
    {{ "text": "<|start_header_id|>ipython<|end_header_id|>[tool_output] Research agent activated. Searching databases... Found: Local watershed impact studies, emission projections, and 3 similar facilities with documented violations.<|python_tag|>search_open_paws({{\"query\":\"{{Get {{Content(where: {{nearText: {{concepts: ['Lancaster County factory farm environmental impact', 'industrial farm violations', 'local watershed contamination']}})}} {{text references}}}}\"}})<|eom_id|>", "author": "tool_output" }},
    {{ "text": "<|start_header_id|>assistant<|end_header_id|>Excellent data gathered. Now, let's automate outreach to local officials while I compose the messaging. Please provide your email draft interface screenshot.<|eot_id|>", "author": "bot" }},
    {{ "text": "<|start_header_id|>user<|end_header_id|><image><|eot_id|>", "author": "human" }},
    {{ "text": "<|start_header_id|>assistant<|end_header_id|>I'll help craft and automate personalized emails to officials.<|python_tag|>computer_use({{\"screenshot\":\"[screenshot]\",\"instructions\":\"Draft email template: 'Dear [Official Name], I am writing regarding the proposed factory farm development in Lancaster County. Our research indicates significant risks to local watersheds, with projected nitrate levels exceeding EPA standards by 47%. Similar facilities have recorded an average of 6.3 environmental violations per year. Additionally, the proposed site lies within 500 feet of historically disadvantaged communities, raising environmental justice concerns. Key findings attached. We request your immediate review and response. Sincerely, [Name]' Set up mail merge for all county commissioners and environmental board members.\"}})<|eom_id|>", "author": "bot" }},
    {{ "text": "<|start_header_id|>ipython<|end_header_id|>[tool_output] Email template created and personalized for 12 local officials. Ready for automated sending.<|eot_id|>", "author": "tool_output" }},
    {{ "text": "<|start_header_id|>assistant<|end_header_id|>Let's now coordinate social media outreach. I'll deploy an agent to analyze optimal posting strategies and manage a coordinated information campaign.<|python_tag|>call_llm_agent_with_tools({{\"role\":\"social_media_coordinator\",\"task_instructions\":\"Design and execute multi-platform social campaign about Lancaster factory farm impact\",\"contextual_data\":{{\"input_text\":\"Focus on environmental justice and local community impact\",\"parameters\":{{\"platforms\":[\"twitter\",\"instagram\",\"facebook\"],\"key_metrics\":[\"local_engagement\",\"official_responses\",\"action_conversions\"]}},\"available_tools\":[{{\"tool_name\":\"computer_use\",\"tool_description\":\"Schedule and post content\",\"tool_instructions\":\"Create posting schedule and engagement tracking\"}}]}})<|eom_id|>", "author": "bot" }},
    {{ "text": "<|start_header_id|>ipython<|end_header_id|>[tool_output] Social media campaign structured: 24 scheduled posts across platforms, automated response tracking, and engagement analysis system activated.<|eot_id|>", "author": "tool_output" }},
    {{ "text": "<|start_header_id|>assistant<|end_header_id|>I've set up automated monitoring of official responses and local engagement. Would you like me to create a real-time dashboard to track campaign metrics and automatically adjust our strategy based on response patterns? We can also set up automated action alerts for supporters when key decision points approach.<|eot_id|>", "author": "bot" }}
  ]
}}

=== LANGUAGE REQUIREMENTS ===

- Only translate the actual conversational content in the "text" fields to {{language}}
- Keep all the following in English:
  * JSON keys and structure
  * Function names and parameters
  * Technical terms in queries (GET, FILTER, WHERE, etc.)
  * Metadata fields and values
  * All code syntax and programming terms
  * Variable names and data structure terms
  * Tool names and technical descriptions
  * Search queries and database operations
  * XML/HTML tags and attributes

The goal is for your output to look extremely similar to the provided examples in terms of structure, formatting, and the way functions are defined and used, with only the actual dialogue text translated to {{language}}. Imagine the examples are a template and your task is to fill in that template with new content relevant to the human's persona and the data provided.

=== FINAL REMINDER ===

IMPORTANT: If you define afunction for tool use, you have to demonstrate how the AI calls and uses that tool within the dialouge. Ensure you output a valid JSON object, paying careful attention to properly escape problematic characters, such as using newlines, quote marks etc. Start your response with an opening curly bracket and end it with a closing one.
"""

    def call_vertex_ai(self, prompt: str) -> str:
        """Make a direct call to Vertex AI"""
        try:
            # Use self.model.generate_content for Gemini
            response = self.model.generate_content(
                contents=[Part.from_text(prompt)], # Assuming prompt is text
                generation_config={
                    "max_output_tokens": self.max_output_tokens,
                    "temperature": self.temperature,
                    "top_p": self.top_p
                }
            )
            return response.text

        except Exception as e:
            print(f"Error calling Vertex AI: {str(e)}")
            return None

    def extract_json_from_text(self, text: str) -> Optional[str]:
        """Extract JSON object from text, handling potential extra content."""
        try:
            # Find the first { and last }
            start = text.find('{')
            end = text.rfind('}')

            if start != -1 and end != -1 and end > start:
                json_str = text[start:end + 1]
                # Test if it's valid JSON
                json.loads(json_str)
                return json_str
            return None
        except:
            return None

    def reformat_data(self, data: str, source_file: str) -> Dict:
        print("\n=== Reformatting Data ===")
        print(f"Data: {data}")
        print(f"Source: {source_file}")

        try:
            human_persona = self.persona_generator.generate_human_persona()
            dialogue_style = self.style_generator.generate_dialogue_style()

            variables = {
                "data": data,
                "source": source_file,
                "dialogue_approaches": ", ".join(dialogue_style['approaches']),
                "dialogue_techniques": ", ".join(dialogue_style['techniques']),
                "human_persona_role": human_persona['role'],
                "human_persona_diet": human_persona['current_lifestyle_diet'],
                "human_persona_advocate": human_persona['advocate_for_animals'],
                "human_persona_age": str(human_persona['age']),
                "human_persona_gender": human_persona['gender'],
                "language": human_persona['language'],
                "human_persona_ethnicity": human_persona['ethnicity'],
                "human_persona_country": human_persona['country'],
                "human_persona_education": human_persona['education_level'],
                "human_persona_income": human_persona['income_level'],
                "human_persona_political": human_persona['political_affiliation'],
                "human_persona_religious": human_persona['religious_affiliation'],
                "human_persona_incrementalist": str(human_persona['incrementalist_vs_abolitionist']),
                "human_persona_institutional": str(human_persona['individual_vs_institutional']),
                "human_persona_intersectional": str(human_persona['solely_on_animal_activism_vs_intersectional']),
                "human_persona_rights": str(human_persona['focus_on_welfare_vs_rights']),
                "human_persona_confrontational": str(human_persona['diplomatic_vs_confrontational']),
                "human_persona_empirical": str(human_persona['intuitive_vs_empirical_effectiveness']),
                "human_persona_values_profit": str(human_persona['values_vs_profit']),
                "human_persona_timeline": str(human_persona['short_term_vs_long_term']),
                "human_persona_innovation": str(human_persona['innovation_vs_tradition']),
                "human_persona_regulation": str(human_persona['pro_regulation_vs_anti_regulation']),
                "human_persona_transparency": str(human_persona['transparency_vs_misinformation']),
                "human_persona_passive": str(human_persona['passive_vs_active'])
            }

            # Format the prompt using the template and variables
            formatted_prompt = self.dialogue_template.format(**variables)

            # Get response directly from Vertex AI
            response_text = self.call_vertex_ai(formatted_prompt)

            #Extract JSON from text
            response_text = self.extract_json_from_text(response_text)

            if response_text:
                try:
                    json_response = json.loads(response_text.strip())

                    # Store the complete persona
                    json_response["questioner_persona"] = human_persona

                    # Store both dialogue style and merge style tags
                    existing_tags = json_response.get("style_tags", [])
                    json_response["dialogue_style"] = {
                        "approaches": dialogue_style['approaches'],
                        "techniques": dialogue_style['techniques'],
                        "style_tags": list(set(existing_tags + dialogue_style['style_tags']))
                    }
                    json_response["style_tags"] = list(set(existing_tags + dialogue_style['style_tags']))

                    print("\n=== Final Enhanced JSON ===")
                    print(json.dumps(json_response, indent=2))
                    return json_response
                except json.JSONDecodeError as e:
                    print(f"JSON parsing error: {str(e)}")
                    print(f"Problematic text: {response_text}")
                    return None

        except Exception as e:
            print(f"Error reformatting data: {str(e)}")
            print(f"Full error: {str(e.__class__.__name__)}: {str(e)}")
            return None

    def process_file_content(self, content: pd.DataFrame) -> List[Dict[str, Any]]:
        """Convert DataFrame content into properly formatted JSONL examples"""
        examples = []

        for idx, row in content.iterrows():
            # Filter valid items
            valid_items = {k: str(v) for k, v in row.items() if pd.notna(v)}
            excluded_terms = ["image", ".png", ".jpg", ".jpeg", ".svg"]
            valid_items = {k: v for k, v in valid_items.items()
                          if not any(term in v.lower() for term in excluded_terms)}

            if valid_items:
                # Create properly formatted messages array
                messages = [
                    {
                        "role": "system",
                        "content": f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\nAnalyze the following data: {' '.join([f'{k}: {v}' for k, v in valid_items.items()])}<|eot_id|>"
                    },
                    {
                        "role": "user",
                        "content": f"<|start_header_id|>user<|end_header_id|>\nWhat insights can we draw from this data?<|eot_id|>"
                    },
                    {
                        "role": "model",
                        "content": f"<|start_header_id|>assistant<|end_header_id|>\nBased on the data...<|eot_id|>"
                    }
                ]

                # Create JSONL example
                example = {
                    "messages": messages
                }

                examples.append(example)

        return examples

    def safe_str_conversion(self, val: Any) -> str:
        if pd.isna(val):
            return ""
        if isinstance(val, (list, dict)):
            return str(val)
        return str(val)

    def save_jsonl(self, examples: List[Dict[str, Any]], output_path: str):
        """Save examples in JSONL format"""
        with open(output_path, 'w') as f:
            for example in examples:
                f.write(json.dumps(example) + '\n')

    def is_valid_content(self, text: str, excluded_terms: List[str]) -> bool:
        if not text or pd.isna(text):
            return False
        return not any(term in text.lower() for term in excluded_terms)

    def load_all_data(self, input_bucket: str) -> List[Dict[str, Any]]:
        print("\nLoading data from random files...")
        all_facts = []
        input_bucket_obj = self.storage_client.get_bucket(input_bucket)

        # Get all blobs including those in subfolders
        blobs = list(input_bucket_obj.list_blobs())

        # Filter for valid file extensions, including files in subfolders
        valid_blobs = [b for b in blobs if b.name.lower().endswith(('.csv', '.json', '.jsonl'))]

        # Randomize the order of blobs
        random.shuffle(valid_blobs)

        selected_blobs = valid_blobs  # Use all valid blobs

        for blob in selected_blobs:
            print(f"\nLoading {blob.name}")
            try:
                # Process JSONL files
                if blob.name.lower().endswith('.jsonl'):
                    rows_processed = 0
                    errors_encountered = 0
                    with blob.open("rt") as f:
                        for line_number, line in enumerate(f, 1):
                            try:
                                if line.strip():
                                    cleaned_line = line.replace('\x00', '')
                                    all_facts.append({
                                        "source_file": blob.name,
                                        "fact_text": cleaned_line.strip()
                                    })
                                    rows_processed += 1
                                    if rows_processed % 1000 == 0:
                                        print(f"Processed {rows_processed} rows from {blob.name}")
                            except Exception as e:
                                errors_encountered += 1
                                print(f"Error processing line {line_number} in {blob.name}: {str(e)}")
                                continue

                    print(f"Completed processing {rows_processed} rows from {blob.name}")
                    if errors_encountered > 0:
                        print(f"Encountered {errors_encountered} errors while processing {blob.name}")

                # Process JSON files
                elif blob.name.lower().endswith('.json'):
                    try:
                        content = blob.download_as_string()
                        cleaned_content = content.decode('utf-8').replace('\x00', '')
                        df = pd.DataFrame([json.loads(cleaned_content)])
                        print(f"Processing {len(df)} rows from {blob.name}")

                        errors_encountered = 0
                        for idx, row in df.iterrows():
                            try:
                                valid_items = {k: str(v) for k, v in row.items() if pd.notna(v)}
                                excluded_terms = ["image", ".png", ".jpg", ".jpeg", ".svg"]
                                valid_items = {k: v for k, v in valid_items.items()
                                            if not any(term in str(v).lower() for term in excluded_terms)}
                                if valid_items:
                                    fact_text = " ".join([f"{k}: {v}" for k, v in valid_items.items()])
                                    all_facts.append({
                                        "source_file": blob.name,
                                        "fact_text": fact_text
                                    })
                            except Exception as e:
                                errors_encountered += 1
                                print(f"Error processing row {idx} in {blob.name}: {str(e)}")
                                continue

                        print(f"Completed processing {len(df)} rows from {blob.name}")
                        if errors_encountered > 0:
                            print(f"Encountered {errors_encountered} errors while processing {blob.name}")

                    except json.JSONDecodeError as e:
                        print(f"JSON parsing error in {blob.name}: {str(e)}")
                        continue

                # Process CSV files
                else:
                    rows_processed = 0
                    errors_encountered = 0
                    try:
                        with blob.open("rt") as file:
                            content = file.read()
                            cleaned_content = content.replace('\x00', '').replace('\r', '\n')

                        lines = cleaned_content.split('\n')
                        if not lines:
                            print(f"No content found in {blob.name}")
                            continue

                        headers = next(csv.reader([lines[0]]))

                        for line_number, line in enumerate(lines[1:], 1):
                            try:
                                if not line.strip():
                                    continue

                                row_values = next(csv.reader([line]))
                                row_dict = {}

                                for i, value in enumerate(row_values):
                                    if i < len(headers):
                                        row_dict[headers[i]] = value
                                    else:
                                        col_name = f"column_{i}"
                                        row_dict[col_name] = value

                                valid_items = {k: str(v) for k, v in row_dict.items()
                                            if v and str(v).strip()}
                                excluded_terms = ["image", ".png", ".jpg", ".jpeg", ".svg"]
                                valid_items = {k: v for k, v in valid_items.items()
                                            if not any(term in str(v).lower() for term in excluded_terms)}

                                if valid_items:
                                    fact_text = " ".join([f"{k}: {v}" for k, v in valid_items.items()])
                                    all_facts.append({
                                        "source_file": blob.name,
                                        "fact_text": fact_text
                                    })
                                    rows_processed += 1

                                if rows_processed % 1000 == 0:
                                    print(f"Processed {rows_processed} rows from {blob.name}")

                            except Exception as e:
                                errors_encountered += 1
                                print(f"Error processing row {line_number} in {blob.name}: {str(e)}")
                                continue

                        print(f"Completed processing {rows_processed} rows from {blob.name}")
                        if errors_encountered > 0:
                            print(f"Encountered {errors_encountered} errors while processing {blob.name}")

                    except Exception as e:
                        print(f"Error reading file {blob.name}: {str(e)}")
                        continue

            except Exception as e:
                print(f"Error processing file {blob.name}: {str(e)}")
                print("Continuing with next file...")
                continue

        print(f"Total facts loaded: {len(all_facts)}")
        return all_facts

    def process_bucket(self, input_bucket: str, output_bucket: str, output_prefix: str = ''):
        """Process bucket contents and save as proper LLaMA 3 formatted dialogues"""
        try:
            print("\nAccessing storage buckets...")
            output_bucket_obj = self.storage_client.get_bucket(output_bucket)

            # Generate timestamp for this processing batch
            batch_timestamp = datetime.now().strftime('%Y%m%d_%H%M%S_%f')

            # Load and process data
            all_facts = self.load_all_data(input_bucket)

            # Randomize the order of facts
            random.shuffle(all_facts)

            for fact_num, fact in enumerate(all_facts):
                print(f"\nReformatting fact {fact_num + 1}/{len(all_facts)}")

                # Get the AI response with multi-turn dialogue
                reformatted = self.reformat_data(fact['fact_text'], fact['source_file'])

                if reformatted:
                    try:
                        # Extract the dialogue array and convert to messages format
                        dialogue = reformatted.get('dialogue', [])
                        messages = []

                        # Convert each dialogue turn to proper LLaMA 3 format
                        for turn in dialogue:
                            if 'author' not in turn or 'text' not in turn:
                                print(f"Skipping malformed turn: {turn}")
                                continue

                            # Handle system message
                            if turn['author'] == 'system':
                                messages.append({
                                    "role": "system",
                                    "content": turn['text']
                                })

                            # Handle human messages
                            elif turn['author'] == 'human':
                                messages.append({
                                    "role": "user",
                                    "content": turn['text']
                                })

                            # Handle bot messages and tool outputs
                            elif turn['author'] in ['bot', 'tool_output']:
                                messages.append({
                                    "role": "assistant" if turn['author'] == 'bot' else "ipython",
                                    "content": turn['text']
                                })

                        # Create unique ID using batch timestamp and fact number
                        unique_id = f"dialogue_{batch_timestamp}_{fact_num:06d}"

                        # Create the JSONL example
                        jsonl_example = {
                            "messages": messages
                            }

                        # Create output filename with detailed path structure
                        output_filename = (
                            f"{output_prefix}"
                            f"batch_{batch_timestamp}/"
                            f"dialogue_{unique_id}.jsonl"
                        )

                        output_blob = output_bucket_obj.blob(output_filename)

                        output_blob.upload_from_string(
                            json.dumps(jsonl_example) + '\n',
                            content_type='application/json'
                        )

                        print(f"Saved reformatted dialogue {unique_id}")

                    except Exception as e:
                        print(f"Error processing dialogue {fact_num}: {str(e)}")
                        continue

        except Exception as e:
            print(f"Error processing bucket: {str(e)}")

def main():
    try:
        reformatter = DataReformatter()
        print("Starting data reformatting...")

        output_bucket_name = 'unranked-training-data'
        output_prefix = 'unranked-text-chat-data/'

        reformatter.process_bucket('accurate-aligned-datasets', output_bucket_name, output_prefix)
        print("Data reformatting complete!")

    except Exception as e:
        print(f"Error in main execution: {str(e)}")
        raise

if __name__ == "__main__":
    main()